In [1]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
import import_ipynb
#변수명 추출
from varname import nameof
#os정렬
import natsort
from tqdm import tqdm
# from standard_func import *
# import standard_func_py as sf
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 진행 여부 함수
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print("실패: 'y' 혹은 'n' 를 입력해 주세요")


# blnk/검색 로직

def blnk_chk(data):
    cnt_chk = len(data)

    chk_nll_list = []
    chk_nll_cnt_list = []
    # null값 체크(개수 파악)
    for i,j in zip(range(len(data.columns)), data.columns):
        indv_cnt = data.loc[data[j] != '', j]
        if len(data[j]) != len(indv_cnt):
            print(j + ": " + str(len(indv_cnt)))
            chk_nll_list.append(j)
            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)
    print("================================")
    if len(chk_nll_list) == 0:
        print("모든 값이 입력되었습니다.")
    elif len(chk_nll_list) > 1 :
        print("blnk(null)이 포함된 컬럼이 탐지되었습니다.")
        print(chk_nll_list)
        print("================================")

    globals()["cnt_chk_out"] = pd.DataFrame(zip(chk_nll_list, chk_nll_cnt_list), columns = ["NULL포함컬럼명", "NULL개수"])
    print(cnt_chk_out)
    
    print("================================")
    print("null 인덱스를 임시 보관합니다.")
    print("================================")
    nll_chk_row = cnt_chk_out.iloc[:,0].to_list()
    for i, j in zip(range(len(nll_chk_row)), nll_chk_row):
        globals()["null_{}".format(i)] = data.loc[data[j] == '', j].index
        print("널인덱스보관_" + str(j) + "(변수명) -> null_{}".format(i))
        
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
#         print(type(col))

# 불필요 컬럼 빼기 - 이름으로 빼기
def brief_col(data, excpt_col_list, name):
    col_list = data.columns
    selected_col_list = [x for x in col_list if x not in excpt_col_list]
    print(selected_col_list)
    globals()["{}".format(name)] = data[selected_col_list]
    
# 불필요 컬럼 빼기 - 순서로 빼기 
# tmp_col_list = [5,6,7,8]
# tmp_tbl_list = [2,3,4,5]
def brief_col_num(data, excpt_col_list, name):
    col_list = data.columns
    col_list_num = [*range(len(col_list))]
    selected_col_list_num = [x for x in col_list_num if x not in excpt_col_list]
    selected_col = col_list[selected_col_list_num]
    print(selected_col)
    globals()["{}".format(name)] = data[selected_col]

# 컬럼명(순서) 선택 및 변경
def cor_col(data, select_col, modify_col, name):
    # 컬럼명(순서) 선택
    globals()["{}".format(name)] = data[select_col]
    # 컬렴명 변경
    globals()["{}".format(name)].columns = modify_col
    
# null 공백처리 함수
def nll_to_blnk(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 Null 공백 처리]")
        for i in data.columns:
            data[i] = data[i].fillna("")
            print(str(i) + ":\t Null 공백 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].fillna("")
        print(col + ":\t  공백 처리 성공")

# 컬럼 고유값 수 비교
def cmpar(data, col1, col2):
    crtr_1 = len(data[col1].unique()) - 1
    crtr_2 = len(data[col2].unique()) - 1
    if crtr_1 == crtr_2:
        print('[정상] 용어명과 분리된용어명의 고유값 수가 같습니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
    else:
        print('[오류] 용어명과 분리된용어명의 고유값 수가 다릅니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
        
# 공백 null처리 함수
def blank_to_nll(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 공백 Null 처리]")
        for i in data.columns:
            data.loc[data[i] == '', i] = None
            print(str(i) + ":\t Null 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data.loc[data[col] == '', col] = None
        print(col + ":\t  Null 처리 성공")
        
def row_nll_rmv(data, name):
    ### 값이 한개라도 있을 경우만 카운트 ###

    data_cnt = len(data)
    notna_list = []
    for i in range(data_cnt):    
        # row 기준 널 체크(값이 한개라도 있을 경우 포함)
        if data.iloc[i].notna().sum() > 1:
            notna_list.append(i)
    globals()["{}".format(name)] = globals()["{}".format(data)].loc[notna_list]
    print(notna_list)
    print(data.shape)
    
# blank row 삭제
def blnk_row_rmv(data, opt, *col):
    if opt == 'all':
        data_cnt = len(data)
        notna_list = []
        for i in range(data_cnt):
            if (data.iloc[i,:] != '').sum() > 1:
                notna_list.append(i)
        globals()["blnk_rmv_all".format(col)] = data.iloc[notna_list]
        print(globals()["blnk_rmv_all".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
    elif opt == 'col':
        data_cnt = len(data)
        notna_list = []
        col = ''.join(col)
        for i in range(data_cnt):
            if (data.loc[i,col] != ''):
                notna_list.append(i)
        globals()["blnk_rmv_{}".format(col)] = data.iloc[notna_list].copy()
        print(globals()["blnk_rmv_{}".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")

In [2]:
# 파이썬 로드후 파일 실행
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)


# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    print('예) path + "\\metadata\\" + folder + "\\01 abnormal_words_correct", "abnormal_words_correct"')
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

### 추가 1) 메타데이터 띄움처리 전 중복 제거..
### 추가 2) 한글명 notnull vs null 분리하여 notnull만 관리(null: 파일 생성 및 기입 요청)
### 추가 3.1) 한글 용어 점검 기능: 한글명+영문명+숫자조합 가능), 그외 모두 제거
### 추가 3.2) 한글 단어 점검 기능: 한글명(숫자조합 가능), 영문명만(숫자조합 가능) 그외 모두 제거
### 추가 4) 테이블기준 상세항목 정의하기.. 

In [3]:
# 1.전체 데이터 로드

In [4]:
opt = 29

# 경로 및 파일명 설정
path = os.path.join(os.getcwd())
# 시스템 리스트업
folder_list = pd.read_excel(path + "\\metadata\\철도공사-업무별-시스템 담당자_표준화용.xlsx", header=1, sheet_name="표준화대상_데이터관리")['폴더관리1']
folder_list = folder_list.map(lambda x : x.split('md ')[-1])
# folder_list = folder_list[folder_list != "00_"].reset_index(drop=True)
folder_list = list(folder_list.unique())
folder = folder_list[opt]
globals()["data_{}".format(folder)] = pd.read_csv(path + "\\metadata\\" + folder + "\\METADATA_UTF8.csv", na_filter = False)
folder

'30_광역철도수입관리시스템'

### 예제)data_30_광역철도수입관리시스템 

In [5]:
input_meta = data_30_광역철도수입관리시스템

# 데이터 소수점 전처리
input_meta.loc[input_meta['DATA_PRECISION'] != ''] 
input_meta.loc[input_meta['DATA_PRECISION'] != '', 'DATA_LENGTH'] =  input_meta['DATA_PRECISION']
# 컬럼명 정리
col_list = ['관리부서코드', '관리부서명', '시스템명', '물리DB명', 'TABLE_NAME', 'COMMENTS',
       'COLUMN_NAME', '컬럼물리명FullName', 'COMMENTS.1', '컬럼설명', '표준도메인그룹명',
       '도메인분류명', 'DATA_TYPE', 'DATA_LENGTH', 'DATA_SCALE']
input_meta = input_meta[col_list]
smpl_col = ["관리부서코드*", "관리부서명*", "시스템명*", "물리DB명*", "테이블물리명*", "테이블한글명*", "컬럼물리명*", "컬럼물리명FullName",
 "컬럼한글명*", "컬럼설명", "표준도메인그룹명*", "도메인분류명*", "데이터타입*", "데이터길이*", "소수점자리"]
input_meta.columns = smpl_col
input_meta = input_meta.fillna('')
input_meta.head()

,관리부서코드*,관리부서명*,시스템명*,물리DB명*,테이블물리명*,테이블한글명*,컬럼물리명*,컬럼물리명FullName,컬럼한글명*,컬럼설명,표준도메인그룹명*,도메인분류명*,데이터타입*,데이터길이*,소수점자리
0,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DUTY_DT,,업무일자,업무일자,,,VARCHAR2,8,
1,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_SETTM_DTIME,,정산일시,정산일시,,,VARCHAR2,14,
2,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_EXTN_STA_NO,,외부역번호,외부역번호,,,NUMBER,4,0
3,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_NO,,장비번호,장비번호,,,NUMBER,3,0
4,,광역철도본부,KOTRIS,US_K,AAM_ADJUST_PMT,역무_이동정산기 정산지불내역,SC_DEV_CLASS_CD,,장비구분코드,장비구분코드,,,VARCHAR2,3,


### 형태 점검(영문명만)

In [6]:
### 변경 now- 이거야!<단순화>
def wd_emoti_rmv(data, col, var, eng_chk_list):
    if var == 'eng':
        data['stwd_eng'] = data[col]
        globals()["{}".format(eng_chk_list)] = []
        for i in tqdm(range(len(data))):
            
            # 대문자 변경
            data.loc[i, 'stwd_eng'] = data.loc[i, 'stwd_eng'].upper()
            # 영문 대문자, 숫자, _ 제한
            cor_eng = re.findall('[A-Z0-9_+]', data.loc[i, 'stwd_eng'])
            print('index: ' + str(i))
            print(data.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_eng'] = ''.join(cor_eng)
            
        # 변경 리스트 저장
        globals()["{}".format(eng_chk_list)] = data[data['컬럼물리명*'] != data['stwd_eng']].index
        cnt = len(data[data['컬럼물리명*'] != data['stwd_eng']].index)
        
        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print(eng_chk_list)
        print("자동 처리: "+ str(cnt))
        print('[자동 처리 완료하였습니다]')

In [7]:
wd_emoti_rmv(input_meta, "컬럼물리명*", 'eng', 'chk_kor_list')

  7%|█████▋                                                                       | 415/5573 [00:00<00:01, 4119.88it/s]

index: 0
SC_DUTY_DT -> SC_DUTY_DT
index: 1
SC_SETTM_DTIME -> SC_SETTM_DTIME
index: 2
SC_EXTN_STA_NO -> SC_EXTN_STA_NO
index: 3
SC_DEV_NO -> SC_DEV_NO
index: 4
SC_DEV_CLASS_CD -> SC_DEV_CLASS_CD
index: 5
SC_PCARD_NO -> SC_PCARD_NO
index: 6
SC_PCARD_CLASS_CD -> SC_PCARD_CLASS_CD
index: 7
SC_PCARD_USER_CLASS_CD -> SC_PCARD_USER_CLASS_CD
index: 8
SC_CHGR_ID -> SC_CHGR_ID
index: 9
SC_DEV_ID -> SC_DEV_ID
index: 10
SC_SETTM_RES_CD -> SC_SETTM_RES_CD
index: 11
SC_PMT_CLASS_CD -> SC_PMT_CLASS_CD
index: 12
SC_CCARD_NO -> SC_CCARD_NO
index: 13
SC_CCARD_APPV_NO -> SC_CCARD_APPV_NO
index: 14
SC_SETTM_BEF_AMT -> SC_SETTM_BEF_AMT
index: 15
SC_SETTM_AMT -> SC_SETTM_AMT
index: 16
SC_SETTM_AFT_AMT -> SC_SETTM_AFT_AMT
index: 17
SC_DUTYR_ID -> SC_DUTYR_ID
index: 18
SC_SEND_SEQ -> SC_SEND_SEQ
index: 19
SC_LAST_DATA_SEQ -> SC_LAST_DATA_SEQ
index: 20
SC_DELAY_DAY_NUM -> SC_DELAY_DAY_NUM
index: 21
SC_RECV_DTIME -> SC_RECV_DTIME
index: 22
SC_DUTY_DT -> SC_DUTY_DT
index: 23
SC_SETTM_DTIME -> SC_SETTM_DTIME
inde

 15%|███████████▍                                                                 | 827/5573 [00:00<00:01, 3134.75it/s]

index: 641
SC_MYBI_PCARD_CHG_AMT -> SC_MYBI_PCARD_CHG_AMT
index: 642
SC_YOUTH_ISS_CNT -> SC_YOUTH_ISS_CNT
index: 643
SC_YOUTH_ISS_AMT -> SC_YOUTH_ISS_AMT
index: 644
SC_YOUTH_ISS_KR_CNT -> SC_YOUTH_ISS_KR_CNT
index: 645
SC_YOUTH_ISS_KR_AMT -> SC_YOUTH_ISS_KR_AMT
index: 646
SC_YOUTH_ISS_EN_CNT -> SC_YOUTH_ISS_EN_CNT
index: 647
SC_YOUTH_ISS_EN_AMT -> SC_YOUTH_ISS_EN_AMT
index: 648
SC_YOUTH_ISS_CN_CNT -> SC_YOUTH_ISS_CN_CNT
index: 649
SC_YOUTH_ISS_CN_AMT -> SC_YOUTH_ISS_CN_AMT
index: 650
SC_YOUTH_ISS_JP_CNT -> SC_YOUTH_ISS_JP_CNT
index: 651
SC_YOUTH_ISS_JP_AMT -> SC_YOUTH_ISS_JP_AMT
index: 652
SC_YOUTH_SETTM_CNT -> SC_YOUTH_SETTM_CNT
index: 653
SC_YOUTH_SETTM_AMT -> SC_YOUTH_SETTM_AMT
index: 654
SC_YOUTH_SETTM_KR_CNT -> SC_YOUTH_SETTM_KR_CNT
index: 655
SC_YOUTH_SETTM_KR_AMT -> SC_YOUTH_SETTM_KR_AMT
index: 656
SC_YOUTH_SETTM_EN_CNT -> SC_YOUTH_SETTM_EN_CNT
index: 657
SC_YOUTH_SETTM_EN_AMT -> SC_YOUTH_SETTM_EN_AMT
index: 658
SC_YOUTH_SETTM_CN_CNT -> SC_YOUTH_SETTM_CN_CNT
index: 659
SC_YOUTH_

 26%|███████████████████▋                                                        | 1445/5573 [00:00<00:01, 2435.67it/s]

...

DSTK_IVNT -> DSTK_IVNT
index: 4270
SUM_IVNT -> SUM_IVNT
index: 4271
DEL_FLG -> DEL_FLG
index: 4272
REG_PS_ID -> REG_PS_ID
index: 4273
REG_DTTM -> REG_DTTM
index: 4274
CHG_PS_ID -> CHG_PS_ID
index: 4275
CHG_DTTM -> CHG_DTTM
index: 4276
KNR_STN_CD -> KNR_STN_CD
index: 4277
CMMN_TP_CD -> CMMN_TP_CD
index: 4278
REG_SQNO -> REG_SQNO
index: 4279
CMID -> CMID
index: 4280
MSG_TYPE -> MSG_TYPE
index: 4281
REQUEST_TIME -> REQUEST_TIME
index: 4282
SEND_DT -> SEND_DT
index: 4283
SEND_TIME -> SEND_TIME
index: 4284
DEST_PHONE -> DEST_PHONE
index: 4285
SEND_PHONE -> SEND_PHONE
index: 4286
MSG_BODY -> MSG_BODY
index: 4287
SEQ_USER -> SEQ_USER
index: 4288
SEND_FLG -> SEND_FLG
index: 4289
DEL_FLG -> DEL_FLG
index: 4290
REG_PS_ID -> REG_PS_ID
index: 4291
REG_DTTM -> REG_DTTM
index: 4292
CHG_PS_ID -> CHG_PS_ID
index: 4293
CHG_DTTM -> CHG_DTTM
index: 4294
EAI_FLG -> EAI_FLG
index: 4295
ABRD_DTTM -> ABRD_DTTM
index: 4296
GOFF_DTTM -> GOFF_DTTM
index: 4297
DPT_EXTR_STN_CD -> DPT_EXTR_STN_CD
index: 4298
ARV_E

 88%|██████████████████████████████████████████████████████████████████▋         | 4892/5573 [00:01<00:00, 2284.01it/s]


REG_DTTM -> REG_DTTM
index: 4718
CHG_PS_ID -> CHG_PS_ID
index: 4719
CHG_DTTM -> CHG_DTTM
index: 4720
OPR_ST_DT -> OPR_ST_DT
index: 4721
OPR_CLS_DT -> OPR_CLS_DT
index: 4722
DPT_EXTR_STN_CD -> DPT_EXTR_STN_CD
index: 4723
ARV_EXTR_STN_CD -> ARV_EXTR_STN_CD
index: 4724
STLB_INTV_DST -> STLB_INTV_DST
index: 4725
SOL_BCH_ACM_DST -> SOL_BCH_ACM_DST
index: 4726
AR_SOL_ACM_ROUT_DST_01 -> AR_SOL_ACM_ROUT_DST_01
index: 4727
AR_SOL_ACM_ROUT_DST_02 -> AR_SOL_ACM_ROUT_DST_02
index: 4728
AR_SOL_ACM_ROUT_DST_03 -> AR_SOL_ACM_ROUT_DST_03
index: 4729
AR_SOL_ACM_ROUT_DST_04 -> AR_SOL_ACM_ROUT_DST_04
index: 4730
AR_SOL_ACM_ROUT_DST_05 -> AR_SOL_ACM_ROUT_DST_05
index: 4731
AR_SOL_ACM_ROUT_DST_06 -> AR_SOL_ACM_ROUT_DST_06
index: 4732
WTPT_BCH_ACM_DST -> WTPT_BCH_ACM_DST
index: 4733
WTPT_AR_ROUT_ACM_DST_01 -> WTPT_AR_ROUT_ACM_DST_01
index: 4734
WTPT_AR_ROUT_ACM_DST_02 -> WTPT_AR_ROUT_ACM_DST_02
index: 4735
WTPT_AR_ROUT_ACM_DST_03 -> WTPT_AR_ROUT_ACM_DST_03
index: 4736
WTPT_AR_ROUT_ACM_DST_04 -> WTPT_AR_ROU

100%|████████████████████████████████████████████████████████████████████████████| 5573/5573 [00:02<00:00, 2496.80it/s]


index: 5192
WIDE_LN_CD -> WIDE_LN_CD
index: 5193
TK_DV_CD -> TK_DV_CD
index: 5194
TRVL_DV_CD -> TRVL_DV_CD
index: 5195
NSTP_PRNB -> NSTP_PRNB
index: 5196
NSTP_PKLO -> NSTP_PKLO
index: 5197
NET_NSTP_PRNB -> NET_NSTP_PRNB
index: 5198
ABRD_PRNB -> ABRD_PRNB
index: 5199
OPR_ISTT_CD -> OPR_ISTT_CD
index: 5200
REG_PS_ID -> REG_PS_ID
index: 5201
REG_DTTM -> REG_DTTM
index: 5202
CHG_PS_ID -> CHG_PS_ID
index: 5203
CHG_DTTM -> CHG_DTTM
index: 5204
STCS_DT -> STCS_DT
index: 5205
DPT_LN_CD -> DPT_LN_CD
index: 5206
ARV_LN_CD -> ARV_LN_CD
index: 5207
STCS_TRM_DV_CD -> STCS_TRM_DV_CD
index: 5208
WHL_CRG_PRNB -> WHL_CRG_PRNB
index: 5209
PS_LN_CRG_PRNB -> PS_LN_CRG_PRNB
index: 5210
WHL_CRG_PKLO -> WHL_CRG_PKLO
index: 5211
PS_LN_CRG_PKLO -> PS_LN_CRG_PKLO
index: 5212
OPR_ISTT_CD -> OPR_ISTT_CD
index: 5213
REG_PS_ID -> REG_PS_ID
index: 5214
REG_DTTM -> REG_DTTM
index: 5215
CHG_PS_ID -> CHG_PS_ID
index: 5216
CHG_DTTM -> CHG_DTTM
index: 5217
STCS_DT -> STCS_DT
index: 5218
DPT_LN_CD -> DPT_LN_CD
index: 521

In [8]:
chk_kor_list

Int64Index([5505, 5506, 5507, 5508, 5509, 5510, 5511, 5512, 5513, 5514, 5515,
            5516, 5517, 5518, 5519, 5520, 5521],
           dtype='int64')

In [9]:
input_meta.loc[chk_kor_list]

,관리부서코드*,관리부서명*,시스템명*,물리DB명*,테이블물리명*,테이블한글명*,컬럼물리명*,컬럼물리명FullName,컬럼한글명*,컬럼설명,표준도메인그룹명*,도메인분류명*,데이터타입*,데이터길이*,소수점자리,stwd_eng
5505,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,테이블유형,,,,,,VARCHAR2,11,,
5506,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,컬럼순서,,,,,,NUMBER,22,,
5507,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,테이블ID,,,,,,VARCHAR2,30,,ID
5508,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,테이블한글명,,,,,,VARCHAR2,4000,,
5509,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,컬럼ID,,,,,,VARCHAR2,30,,ID
5510,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,컬럼한글명,,,,,,VARCHAR2,4000,,
5511,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,자료형,,,,,,VARCHAR2,106,,
5512,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,길이,,,,,,VARCHAR2,81,,
5513,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,NULL허용여부,,,,,,VARCHAR2,1,,NULL
5514,,광역철도본부,KOTRIS,US_K,VW_OBJECTINFO_K,,기본값,,,,,,LONG,0,,


### 중복 나열

In [64]:
# 인터페이스 No
input_test = input_meta.copy()
input_test['No'] = [*range(len(input_test))]
input_test.columns

Index(['관리부서코드*', '관리부서명*', '시스템명*', '물리DB명*', '테이블물리명*', '테이블한글명*', '컬럼물리명*',
       '컬럼물리명FullName', '컬럼한글명*', '컬럼설명', '표준도메인그룹명*', '도메인분류명*', '데이터타입*',
       '데이터길이*', '소수점자리', 'stwd_eng', 'No'],
      dtype='object')

In [65]:
# 인덱스 DF 초기화
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']

In [66]:
# 영문명 분리
input_split_eng = pd.DataFrame(input_test['컬럼물리명*'].str.split('_').tolist()).fillna("")
input_split_eng

,0,1,2,3,4,5,6
0,SC,DUTY,DT,,,,
1,SC,SETTM,DTIME,,,,
2,SC,EXTN,STA,NO,,,
3,SC,DEV,NO,,,,
4,SC,DEV,CLASS,CD,,,
...,...,...,...,...,...,...,...
5568,STN,CD3,,,,,
5569,TM,MIN,,,,,
5570,DATA,TYPE,,,,,
5571,UPWN,DV,CD,,,,


In [67]:
stwd_eng6

,index,stwd_eng6,num_eng6,dupl_list_eng6
0,0,,,
1,1,,,
2,2,,,
3,3,,,
4,4,,,
...,...,...,...,...
5568,5568,,,
5569,5569,,,
5570,5570,,,
5571,5571,,,


In [78]:
# 영문명 컬럼 수
eng_firts_cnt = len(input_split_eng.columns)

# 컬럼명 입력
col_list_eng = []
for i in range(len(input_split_eng.columns)):
    col_list_eng.append("stwd_eng{}".format(i))
input_split_eng.columns = col_list_eng
input_split_eng['stwd_index'] = input_split_eng.index
input_split_eng

,stwd_eng0,stwd_eng1,stwd_eng2,stwd_eng3,stwd_eng4,stwd_eng5,stwd_eng6,stwd_eng7,stwd_eng8,stwd_eng9,stwd_eng10,stwd_eng11,stwd_eng12,stwd_eng13,stwd_eng14,stwd_index
0,SC,DUTY,DT,,,,,0,None,None,None,None,None,None,None,0
1,SC,SETTM,DTIME,,,,,1,None,None,None,None,None,None,None,1
2,SC,EXTN,STA,NO,,,,2,None,None,None,None,None,None,None,2
3,SC,DEV,NO,,,,,3,None,None,None,None,None,None,None,3
4,SC,DEV,CLASS,CD,,,,4,None,None,None,None,None,None,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5568,STN,CD3,,,,,,5568,None,None,None,None,None,None,None,5568
5569,TM,MIN,,,,,,5569,None,None,None,None,None,None,None,5569
5570,DATA,TYPE,,,,,,5570,None,None,None,None,None,None,None,5570
5571,UPWN,DV,CD,,,,,5571,None,None,None,None,None,None,None,5571


In [69]:
# 영문 개수 세기
dupl_list_eng = []
dupl_cnt_eng = []
for i in range(eng_firts_cnt):
    globals()["dupl_list_eng{}".format(i)] = []
    for j in range(len(input_split_eng["stwd_eng{}".format(i)])):
        dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
        globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)
for i in range(eng_firts_cnt):
    globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
    globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]

# # 컬럼명,index, 단어위치 추가
# for i in range(eng_firts_cnt):
#     input_split_eng["num_eng{}".format(i)] = "num_eng{}".format(i)
#     globals()["stwd_eng{}".format(i)] = pd.concat(
#             [
#             df_index.fillna(""), 
#             pd.DataFrame(input_split_eng["stwd_eng{}".format(i)].fillna("")),
#             pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
#             globals()["dupl_list_eng{}".format(i)].fillna("")
#             ], 
#             axis=1)

In [76]:
input_split_eng

,stwd_eng0,stwd_eng1,stwd_eng2,stwd_eng3,stwd_eng4,stwd_eng5,stwd_eng6,stwd_index,dupl_list_kor0,dupl_list_kor1,dupl_list_kor2,dupl_list_kor3,dupl_list_kor4,dupl_list_kor5,dupl_list_kor6
0,SC,DUTY,DT,,,,,0,None,None,None,None,None,None,None
1,SC,SETTM,DTIME,,,,,1,None,None,None,None,None,None,None
2,SC,EXTN,STA,NO,,,,2,None,None,None,None,None,None,None
3,SC,DEV,NO,,,,,3,None,None,None,None,None,None,None
4,SC,DEV,CLASS,CD,,,,4,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5568,STN,CD3,,,,,,5568,None,None,None,None,None,None,None
5569,TM,MIN,,,,,,5569,None,None,None,None,None,None,None
5570,DATA,TYPE,,,,,,5570,None,None,None,None,None,None,None
5571,UPWN,DV,CD,,,,,5571,None,None,None,None,None,None,None


In [71]:
# 컬럼수 일치 점검
col_cnt_eng = len(col_list_eng)
# if len(col_list_kor) == len(col_list_eng):
#     print("한글컬럼수와 영문컬럼수 일치")
#     print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
# else:
#     print("한글컬럼수와 영문컬럼수 불일치")
#     print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))                                                       
# print(col_list_eng)
# stwd_eng0


In [72]:
col_cnt_diff_list = list(set(range(col_cnt_eng)) - set(range(0)))
col_cnt_diff_list

[0, 1, 2, 3, 4, 5, 6]

In [73]:
col_cnt_eng

7

In [74]:
input_split_kor

,stwd_kor0,num_kor0,stwd_kor1,num_kor1,stwd_kor2,num_kor2,stwd_kor3,num_kor3,stwd_kor4,num_kor4,stwd_kor5,num_kor5,stwd_kor6,num_kor6


In [86]:
# 영문컬럼이 더 많을 경우
col_cnt = col_cnt_eng
# input_split_kor = pd.DataFrame()

for i in range(col_cnt_eng):
    input_split_kor['stwd_kor{}'.format(i)] = None
    input_split_kor['num_kor{}'.format(i)] = None
    input_split_eng['dupl_list_kor{}'.format(i)] = None
    globals()['stwd_kor{}'.format(i)] = pd.concat(
        [
            df_index,
            pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)]),
            pd.DataFrame(input_split_kor["num_kor{}".format(i)]),
            pd.DataFrame(input_split_eng["dupl_list_kor{}".format(i)])
        ],
        axis = 1
    ).fillna("")
col_cnt_kor = col_cnt
print("컬럼수 일치작업 후:" + str(col_cnt))  
stwd_kor0
stwd_eng0

컬럼수 일치작업 후:7


,index,stwd_eng0,num_eng0,dupl_list_eng0
0,0,,,
1,1,,,
2,2,,,
3,3,,,
4,4,,,
...,...,...,...,...
5568,5568,,,
5569,5569,,,
5570,5570,,,
5571,5571,,,


In [87]:
# n x m 정사각형 DafaFrame 점검
for i in range(col_cnt_kor):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_kor = round(np.mean(avg), 2)
for i in range(col_cnt_eng):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_eng = round(np.mean(avg), 2)
stwd_total_cnt = int(avg_avg_eng)
if avg_avg_kor == avg_avg_eng:
    print("한글 영문 row수 일치: 평균=" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))
else:
    yes_or_no("한글명과 영문명의 row count가 일치하지 않습니다. 계속 진행하시겠습니까?")
    print("한글 영문 row수 불일치: 평균" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))


한글 영문 row수 일치: 평균=5573.0
한글평균:5573.0, 영문평균:5573.0, 최종count:5573


In [88]:
# 중복검사를 위한 left join
for i in range(col_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )
print(col_cnt)
# stwd_total9

7


In [89]:
stwd_total0

,index,stwd_kor0,num_kor0,dupl_list_kor0,stwd_eng0,num_eng0,dupl_list_eng0
0,0,,,,,,
1,1,,,,,,
2,2,,,,,,
3,3,,,,,,
4,4,,,,,,
...,...,...,...,...,...,...,...
5568,5568,,,,,,
5569,5569,,,,,,
5570,5570,,,,,,
5571,5571,,,,,,


In [90]:
# 일렬 세우기

# unique_stwd 초기화 
stwd_total = pd.DataFrame()

## 컬럼명 바꾸고  concat
for i in range(col_cnt):
    globals()["stwd_total{}".format(i)].columns = [
        'index', 'stwd_kor', 'num_kor', 'dupl_list_kor', 'stwd_eng','num_eng', 'dupl_list_eng'
    ]
    stwd_total = pd.concat([stwd_total, globals()["stwd_total{}".format(i)]], axis = 0, ignore_index = True)
print(stwd_total.shape)
stwd_total

(39011, 7)


,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng
0,0,,,,,,
1,1,,,,,,
2,2,,,,,,
3,3,,,,,,
4,4,,,,,,
...,...,...,...,...,...,...,...
39006,5568,,,,,,
39007,5569,,,,,,
39008,5570,,,,,,
39009,5571,,,,,,


In [ ]:
len(input_meta)

In [ ]:
input_meta.duplicated("컬럼물리명*")

In [ ]:
test = input_meta.groupby('컬럼물리명*').head(1)
test.head()

In [ ]:
# 2.중복제거 후 형태이상단어 검색

In [ ]:
input_meta.groupby('컬럼한글명*').head(1)

In [ ]:
# 모든 테이블 로드
for i in folder_list:
    print(i)
    globals()["data_{}".format(i)] = pd.read_csv(path + "\\metadata\\" + i + "\\METADATA_UTF8.csv", na_filter = False)
#     globals()["data_{}".format(i)] = pd.DataFrame()

# 형태이상 단어 탐색

In [ ]:
wd_emoti_rmv(input_meta, "컬럼한글명*", 'kor', 'chk_kor_list')

In [ ]:
# 경로 및 파일명 설정
path = os.path.join(os.getcwd())
folder_list = pd.read_excel(path + "\\metadata\\폴더명.xlsx")['폴더명']
folder = folder_list[10]
folder

In [ ]:
# 파일 불러오기
data_name = "METADATA_UTF8.csv"
sheet_name = ""
input_meta = pd.read_csv(path + "\\metadata\\" + folder + "\\" + data_name, na_filter=False)
print(input_meta.shape)
input_meta.head(5)

In [ ]:
wd_emoti_rmv(input_meta, "컬럼한글명*", 'kor', 'chk_kor_list')

### 한글명 미입력 처리 제외 안하고 그냥 가기

In [ ]:
# input_meta = input_meta[input_meta['테이블한글명*'] != '']
print(input_meta.shape)
input_meta.head()

### 중복제거후 수정

In [ ]:
len(input_meta.groupby(['stwd_kor']).head(1)['stwd_kor'])

In [ ]:
input_meta['stwd_spce'] = input_meta['stwd_kor']
input_meta.head()

In [ ]:
input_meta.groupby(['stwd_kor']).head(1)[['stwd_kor', 'stwd_spce']]

In [ ]:
# read & load

# 파이썬 로드후 파일 실행
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)


# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    print('예) path + "\\metadata\\" + folder + "\\01 abnormal_words_correct", "abnormal_words_correct"')
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

In [ ]:
input_meta['테이블한글명*'].unique()

In [ ]:
input_meta['stwd_kor'].unique()

In [ ]:
save_grip_file(input_meta.loc[chk_kor_list], )

In [ ]:
print(len(input_meta.loc[chk_kor_list]))
input_meta.loc[chk_kor_list]

### 한글명 null 없애기(수기로)

In [ ]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣_+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

In [ ]:
# 표준용어 구분자 변환 점검
print(len('가 나 다'.split(" ")))
'가 나 다'.split(" ")

In [ ]:
'가 나 다'.count(" ")

# 한글명은 notNull이어야 함

작업내용:
1. 중복제거(unique): 작업 효율성 제고
2. 개수 매칭 -> (모든 값 채우기)
3. 순서 매칭: ID 부여, 한글/영문
4. 재조립( 한글명 기준)

In [ ]:

테이블 한글명 + 컬럼 한글명 한줄 생성
도메인 한글명 - > 도메인명 생성

# 용어분리

In [ ]:
# 파일 불러오기
data_name = "input_test_KORAIL"
sheet_name = "테이블컬럼 입력항목 통계조사"
input_path = os.path.join(os.getcwd(), "input\\")
output_path = os.path.join(os.getcwd(), "output\\")
input_meta = pd.read_excel(input_path + data_name +".xlsx", sheet_name=sheet_name, header=0, na_filter=False)

In [ ]:
# 테이블 따로(명명규칙)
input_meta['분리된테이블한글명'] = input_meta['테이블한글명*']

# 컬럼 따로(명명규칙)
input_meta['분리된컬럼한글명'] = input_meta['컬럼한글명*']

# 컬럼명부터

In [ ]:
input_meta_sepa_col = pd.DataFrame(input_meta['분리된컬럼한글명'])
input_meta_sepa_col

In [ ]:
input_meta_sepa_col_unq = input_meta_sepa_col['분리된컬럼한글명'].unique()
input_meta_sepa_col_unq

In [ ]:
zip(input_meta_sepa_col_unq, input_meta_sepa_col_unq)

In [ ]:
a = pd.DataFrame(zip(input_meta_sepa_col_unq,input_meta_sepa_col_unq), columns = ['원본', '구분자입력'])
a


In [ ]:
input_meta_sepa_col

In [ ]:
input_meta_sepa_col.merge(on=a, how='left', left_on='분리된컬럼한글명', right_on='원본')

In [ ]:
pd.DataFrame([input_meta_sepa_col_unq, input_meta_sepa_col_unq])

In [ ]:
pd.DataFrame(zip(input_meta_sepa_tbl_uniq, input_meta_sepa_tbl_uniq), columns = ['원본', '구분자입력'])

In [ ]:
pd.DataFrame([input_meta_sepa_tbl_uniq, input_meta_sepa_tbl_uniq])

In [ ]:
input_meta_sepa_tbl

In [ ]:
input_meta_sepa_tbl.iloc[:,0]

In [ ]:
pd.DataFrame(columns = input_meta_sepa_tbl.columns)

In [ ]:
input_meta_sepa_tbl = input_meta_sepa_tbl.fillna('')

In [ ]:
emoti_rmv(input_meta_sepa_tbl, '분리된테이블한글명', 'kor')

In [ ]:
input_meta_sepa_tbl['stwd_kor'] = input_meta_sepa_tbl['분리된테이블한글명']
input_meta_sepa_tbl['stwd_kor']

In [ ]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣_+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

In [ ]:
globals()["{}".format(nameof(stwd_total))] = rw_data
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):
        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [ ]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')
        

        print(data)
        print(1)

In [ ]:
globals()["{}".format(nameof(stwd_total))] = rw_data
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):
        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [ ]:
# # 파일 불러오기
# sheet_name = "테이블컬럼정의서"
# input_path = os.path.join(os.getcwd(), "input\\")
# output_path = os.path.join(os.getcwd(), "output\\")
# input_meta = pd.read_excel(input_path+"표준화 INPUT 정의서.xlsx", sheet_name=sheet_name, header=0, na_filter=False)

# # null string ""로 변환
# for i in range(len(input_meta.columns)):
#     input_meta[input_meta.columns[i]] = input_meta[input_meta.columns[i]].fillna("")
# input_meta['기관ID'] = input_meta['물리DB명*'] + "_" + input_meta['관리부서명*']
# print(input_meta.shape)
# input_meta.head(5)

In [ ]:
input_meta

### 값 검색 로직

In [ ]:
# 널 체크
blnk_chk(input_meta)

In [ ]:
input_meta_tbl

In [ ]:
len(input_meta_tbl.columns)

In [ ]:
# 테이블 유니크 선벌(그룹바이)
input_meta_tbl = input_meta.groupby(['테이블물리명*', '테이블한글명*']).head(1)
# 불필요 값 없애기

for i in range(6,14):
    input_meta_tbl.iloc[:,i] = ''
input_meta_tbl

In [ ]:
# 불필요 컬럼 빼기
excpt_col = ['컬럼물리명*', '컬럼영문약어명*', '컬럼한글명*', '분리된컬럼한글명*']
excpt_tbl = ['테이블물리명*', '테이블영문약어명*', '테이블한글명*', '분리된테이블한글명*']

# 컬럼명 추림
brief_col(input_meta_tbl, excpt_col, 'pick_tbl')
brief_col(input_meta, excpt_tbl, 'pick_col')

# 용어명 컬럼 추가
pick_tbl['용어명'] = pick_tbl['테이블한글명*']
pick_col['용어명'] = pick_col['컬럼한글명*']
pick_tbl

In [ ]:
# 컬럼명 변경
modify_col = ['물리DB명*', '관리부서명*', '테이블물리명*', '영문약어명', '테이블한글명*', '용어명 분리', 
              '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', '용어명']
cor_col(pick_tbl, pick_tbl.columns, modify_col, 'pick_tbl')
modify_col = ['물리DB명*', '관리부서명*', '컬럼물리명*', '영문약어명', '컬럼한글명*', '용어명 분리', 
              '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', '용어명']
cor_col(pick_col, pick_col.columns, modify_col, 'pick_col')

# TBL/COL 기록
pick_col["TBL/COL"] = "COL"
pick_tbl["TBL/COL"] = "TBL"

In [ ]:
# null 제거 및 컬럼(순서) 선택
pick_total = pd.concat([pick_col, pick_tbl]).fillna("")
pick_total_col = ['물리DB명*', '관리부서명*',  '테이블물리명*', '테이블한글명*', '컬럼물리명*', '컬럼한글명*', 
                  '용어명', '용어명 분리', '영문약어명', 
                  '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', 'TBL/COL']
cor_col(pick_total, pick_total_col, pick_total_col, 'pick_total')
# 인덱스 초기화
pick_total = pick_total.reset_index(drop=True)
print(pick_total.shape)
pick_total.head()

#### . 파일 불러오기

In [ ]:
# # 파일 불러오기
# # test_emoti_dup_korDup
# sheet_name = "input_test"
# input_path = os.path.join(os.getcwd(), "input\\")
# output_path = os.path.join(os.getcwd(), "output\\")
# # input_test = pd.read_excel(input_path+"test_emoti_dup_korDup.xlsx", sheet_name=sheet_name, header=0)
# input_test = pd.read_excel(input_path+"test_final.xlsx", sheet_name=sheet_name, header=0)
# # test_final.xlsx
# # null 제거
# for i in range(len(input_test.columns)):
#     input_test[input_test.columns[i]] = input_test[input_test.columns[i]].fillna("")
# print(input_test.shape)
# input_test.head(5)

### 추가 1) 메타데이터 띄움처리 전 중복 제거..
### 추가 2) 한글명 notnull vs null 분리하여 notnull만 관리(null: 파일 생성 및 기입 요청)
### 추가 3.1) 한글 용어 점검 기능: 한글명+영문명+숫자조합 가능), 그외 모두 제거
### 추가 3.2) 한글 단어 점검 기능: 한글명(숫자조합 가능), 영문명만(숫자조합 가능) 그외 모두 제거
### 추가 4) 테이블기준 상세항목 정의하기.. 

# 2. 용어 분리

### 용어명 변경

In [ ]:
input_meta_spce = pick_total.copy()
input_meta_spce['영문약어명'] = ''
input_meta_spce['용어명 분리'] = ''
input_meta_spce.head()

In [ ]:
rmv_spce(input_meta_spce, 'col', '용어명')
rmv_spce(input_meta_spce, 'col', '영문약어명')
rmv_spce(input_meta_spce, 'col', '용어명 분리')

In [ ]:
컬럼한글명 유니크!! - 용어명이 KEY값임
out 후 적용
수정 후 적용 - 한글명기준(인덱스 무시)
중복 검사
1. 컬럼한글명 유니크 수 == 컬럼한글명&&분리된컬럼한글명 유니크 수
# ->단어 변환 후 검사 노!!!!!!!!! 여기서는 1차 분리임!!!!!!!!!

검사?

### 컬럼한글명 유니크 생성

In [ ]:
input_meta_dict = input_meta_spce.groupby('용어명').head(1)[['용어명','용어명 분리', '영문약어명', 'TBL/COL']].reset_index(drop=True)
print(input_meta_dict.shape)
input_meta_dict.head()

In [ ]:
# 저장
input_meta_dict.to_excel(output_path + 'input_meta_space.xlsx')

In [ ]:
input_meta_dict = pd.read_excel(output_path + 'input_meta_space.xlsx', header=0, index_col=0, na_filter=False)
print(input_meta_dict.shape)
input_meta_dict.head()

In [ ]:
cmpar(input_meta_dict, '용어명', '용어명 분리')
cmpar(input_meta_dict, '용어명', '영문약어명')

# 분리 시작

In [ ]:
# 인터페이스 No
input_test = pick_total.copy()
input_test['No'] = [*range(len(input_test))]
input_test.columns

In [ ]:
# 인덱스 DF 초기화
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']

In [ ]:
# 용어명 분리

# 용어명 분리
input_split_kor = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist()).fillna("")

# 영문명 없을 경우 대비하여 cnt 집계
kor_firts_cnt = len(input_split_kor.columns)

# 컬럼명 입력
col_list_kor = []
for i in range(len(input_split_kor.columns)):
    col_list_kor.append("stwd_kor{}".format(i))
input_split_kor.columns = col_list_kor

# 한글 개수 세기
dupl_list_kor = []
dupl_cnt_kor = []
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = []
    for j in range(len(input_split_kor["stwd_kor{}".format(i)])):
        dupl_list_kor = input_split_kor["stwd_kor{}".format(i)].to_list().count(input_split_kor["stwd_kor{}".format(i)][j])
        globals()["dupl_list_kor{}".format(i)].append(dupl_list_kor)
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_kor{}".format(i)])], axis = 0)
    globals()["dupl_list_kor{}".format(i)].columns = ["dupl_list_kor{}".format(i)]
    
# 컬럼명,index, 단어위치 추가
for i in range(len(input_split_kor.columns)):
    input_split_kor["num_kor{}".format(i)] = "num_kor{}".format(i)
    globals()["stwd_kor{}".format(i)] = pd.concat(
            [
            df_index.fillna(""), 
            pd.DataFrame(input_split_kor["stwd_kor{}".format(i)]).fillna(""),
            pd.DataFrame(input_split_kor["num_kor{}".format(i)]).fillna(""),
            globals()["dupl_list_kor{}".format(i)].fillna("")
            ], 
            axis=1)
print(col_list_kor)
print(kor_firts_cnt)
# stwd_kor9
# dupl_list_kor0

In [ ]:
# 영문명 분리

# 영문명이 하나라도 입력돼 있을 경우
if len(input_test['영문약어명']) > 0:
    print("영문명 컬럼 수 1개 이상:"+str((input_test['영문약어명']=="").sum()))

    # 영문명 분리
    input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist()).fillna("")
    
    # 영문명 컬럼 수
    eng_firts_cnt = len(input_split_eng.columns)
    
    # 컬럼명 입력
    col_list_eng = []
    for i in range(len(input_split_eng.columns)):
        col_list_eng.append("stwd_eng{}".format(i))
    input_split_eng.columns = col_list_eng
    
    # 영문 개수 세기
    dupl_list_eng = []
    dupl_cnt_eng = []
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = []
        for j in range(len(input_split_eng["stwd_eng{}".format(i)])):
            dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
            globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
        globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]
        
    # 컬럼명,index, 단어위치 추가
    for i in range(eng_firts_cnt):
        input_split_eng["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)] = pd.concat(
                [
                df_index.fillna(""), 
                pd.DataFrame(input_split_eng["stwd_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                globals()["dupl_list_eng{}".format(i)].fillna("")
                ], 
                axis=1)
        
# 영문명이 없을 경우
elif len(input_test['영문약어명']) == 0:
    print("영문명 컬럼 수 0개:"+str((input_test['영문약어명']=="").sum()))
    
    col_list_eng = []
    for i in range(kor_firts_cnt):
        globals()["stwd_eng{}".format(i)] = pd.DataFrame(columns = ["stwd_eng{}".format(i)])
        globals()["num_eng{}".format(i)] = pd.DataFrame(columns = ["num_eng{}".format(i)])
        globals()["dupl_list_eng{}".format(i)] = pd.DataFrame(columns = ["dupl_list_eng{}".format(i)])
        globals()["stwd_eng{}".format(i)] = pd.concat(
            [
                df_index.fillna(""), 
                globals()["stwd_eng{}".format(i)].fillna(""),
                globals()["num_eng{}".format(i)].fillna(""),
                globals()["dupl_list_eng{}".format(i)].fillna(""),
            ]
            , axis = 1
        )
        globals()["stwd_eng{}".format(i)]["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)]["dupl_list_eng{}".format(i)] = len(globals()["stwd_eng{}".format(i)])
        col_list_eng.append("stwd_eng{}".format(i))
        
else:
    print("한글컬럼, 영문커럼 예외 사항")

# 컬럼수 일치 점검
col_cnt_kor = len(col_list_kor)
col_cnt_eng = len(col_list_eng)
if len(col_list_kor) == len(col_list_eng):
    print("한글컬럼수와 영문컬럼수 일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
else:
    print("한글컬럼수와 영문컬럼수 불일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))                                                       
print(col_list_eng)
stwd_eng0

In [ ]:
# 한글/영문 컬럼 수 일치시키기

# 한글컬럼과 영문컬럼 수 일치할 경우
if col_cnt_kor == col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = None
    col_cnt_diff = None
    print('한글컬럼과 영문컬럼 수 일치')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
    
# 한글컬럼이 더 많을 경우
elif col_cnt_kor > col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = list(set(range(col_cnt_kor)) - set(range(col_cnt_eng)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        input_split_eng['dupl_list_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["dupl_list_eng{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('한글컬럼이 더 많은 싱태')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  

# 영문컬럼이 더 많을 경우
elif col_cnt_kor < col_cnt_eng:
    col_cnt = col_cnt_eng
    col_cnt_diff_list = list(set(range(col_cnt_eng)) - set(range(col_cnt_kor)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        input_split_eng['dupl_list_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)].fillna("")).
                pd.DataFrame(input_split_eng["dupl_list_kor{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('영문컬럼이 더 많은 싱태')       
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
stwd_eng3

In [ ]:
# n x m 정사각형 DafaFrame 점검
for i in range(col_cnt_kor):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_kor = round(np.mean(avg), 2)
for i in range(col_cnt_eng):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_eng = round(np.mean(avg), 2)
stwd_total_cnt = int(avg_avg_eng)
if avg_avg_kor == avg_avg_eng:
    print("한글 영문 row수 일치: 평균=" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))
else:
    yes_or_no("한글명과 영문명의 row count가 일치하지 않습니다. 계속 진행하시겠습니까?")
    print("한글 영문 row수 불일치: 평균" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))

In [ ]:
# 중복검사를 위한 left join
for i in range(col_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )
print(col_cnt)
# stwd_total9

In [ ]:
# 일렬 세우기

# unique_stwd 초기화 
stwd_total = pd.DataFrame()

## 컬럼명 바꾸고  concat
for i in range(col_cnt):
    globals()["stwd_total{}".format(i)].columns = [
        'index', 'stwd_kor', 'num_kor', 'dupl_list_kor', 'stwd_eng','num_eng', 'dupl_list_eng'
    ]
    stwd_total = pd.concat([stwd_total, globals()["stwd_total{}".format(i)]], axis = 0, ignore_index = True)
print(stwd_total.shape)
stwd_total

In [ ]:
nll_chk(stwd_total)

# 3. 이상값 수정

### 3.1. 한글명에 한글과 영문대문자, 숫자 이외 표시 및 삭제 로직

In [ ]:
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):

        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [ ]:
# 데이터 검토
ans = yes_or_no("데이터 검토를 실행 하시겠습니까?")
if ans == True:
    emoti_chk('kor')
    emoti_chk('eng')
elif ans == False:
    print("계속 진행하겠습니다")

### 3.2. 특수문자 제거 참고

In [ ]:
# 변경 체크 로직

s1 = '고구마acAB1!$%@! 32'
s2 = '깎ㅉ니'
s3 = "RR깍찌니12"
s4 = "123깍찌니aS"
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s1))
print(str(len(re.findall('[A-Z0-9가-힣+]', s1))) + " vs " + str(len(list(s1))))
print(len(re.findall('[A-Z0-9가-힣+]', s1)) == len(list(s1)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s2))
print(str(len(re.findall('[A-Z0-9가-힣+]', s2))) + " vs " + str(len(list(s2))))
print(len(re.findall('[A-Z0-9가-힣+]', s2)) == len(list(s2)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s3))
print(str(len(re.findall('[A-Z0-9가-힣+]', s3))) + " vs " + str(len(list(s3))))
print(len(re.findall('[A-Z0-9가-힣+]', s3)) == len(list(s3)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s4))
print(str(len(re.findall('[A-Z0-9가-힣+]', s4))) + " vs " + str(len(list(s4))))
print(len(re.findall('[A-Z0-9가-힣+]', s4)) == len(list(s4)))
print("------------------------------------------------------")

In [ ]:
path_dict = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
dict_insp = pd.read_csv(path_dict + '/dict_total/' + 'dict_insp.csv', index_col=0, na_filter = False)
print(dict_insp.shape)
dict_insp

In [ ]:
dict_list = dict_insp.iloc[:,0]
dict_list

In [ ]:
dict_insp[dict_insp['어휘'] == '등급']

In [ ]:
path_dict = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
dict_insp_oneMore = pd.read_csv(path_dict + '/dict_total/' + 'dict_expn_oneMore.csv', index_col=0, na_filter=False)
print(dict_insp_oneMore.shape)
dict_insp_oneMore

In [ ]:
dict_insp[dict_insp['어휘'] =='라']

# 국어사전과 ISIN 비교

# 사전에 없는 단어 목록 분리

In [ ]:
# 사전에 있는 단어 목록
dict_is = stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘'])]
dict_is

In [ ]:
# 사전에 없는 단어 목록
dict_nan = stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘']) == False]
dict_nan

In [ ]:
dict_nan_unq = pd.DataFrame(dict_nan['stwd_kor'].unique(), columns = ['bf_stwd_kor'])
dict_nan_unq['af_stwd_kor'] = ''
dict_nan_unq

In [ ]:
# dict_isin 초기화
def dict_isin_init(data):
    data['dict_chk'] = 'N'
    data['notin_wrn'] = ''
    print('dict_chk 컬럼이 초기화 되었습니다.')

dict_isin_init(dict_nan_unq)
dict_nan_unq

# 저장

In [ ]:
# 저장
dict_nan_unq.to_excel(output_path + "dict_nan_unq.xlsx")

# 파이썬 로드, 파일열기 및 수기수정

In [ ]:
# 파이썬 로드후 파일로드
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)

In [ ]:
# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

In [ ]:
# 로드 및 파일실행
read_grip_file(dict_nan_unq, output_path, "dict_nan_unq")
dict_nan_unq

1. dict_chk = 'Y'
2. af_stwd_kor = isin(사전List)
3. ''

In [ ]:
# 가입뿅 테스트
dict_nan_unq.loc[dict_nan_unq['af_stwd_kor'] == '가입', 'af_stwd_kor'] = '가입뿅'
dict_nan_unq

# (수기수정 완료했다면) 로드 후=점검~~~~~~~~

In [ ]:
def chk_dict_isin(data, c_list):
    chk_Y = data.loc[data['dict_chk']=='Y', 'dict_chk']
    chk_N = data.loc[data['dict_chk']=='N', 'dict_chk']
    # 모두 dict_chk(Y) 일 경우통과
    if chk_Y.count() == data['dict_chk'].count():
        print("모두 수정되었습니다 "+'dict_chk'+" count(Y): " + str(chk_Y.count()))
         # 모두 notin_wrn(Y) 일 경우통과
        chk_notin_wrn = data.loc[~data['af_stwd_kor'].isin(c_list), 'notin_wrn'] 
        data['notin_wrn'] = ''
        if len(chk_notin_wrn) > 0:
            data.loc[~data['af_stwd_kor'].isin(c_list), 'notin_wrn'] = 'Y'
            print("[경고] 사전에 등록되어 있지 않은 단어가 존재합니다")
            print(data.loc[data['notin_wrn'] == 'Y', 'af_stwd_kor'])
        else:
            print("모든 단어가 사전에 등록되어 있습니다.")
    # dict_chk(N) 이 한개라도 있을 경우 반려
    elif chk_N.count() > 0:
        print("[경고] 아직 수정하지 않은 데이터가 있습니다 "+'dict_chk'+" count(N): " + str(chk_N.count()))

In [ ]:
# 가입뿅 테스트
dict_nan_unq.loc[dict_nan_unq['af_stwd_kor'] == '가입뿅', 'af_stwd_kor'] = '가입'
dict_nan_unq

In [ ]:
chk_dict_isin(dict_nan_unq, dict_list)
dict_nan_unq

In [ ]:
# 로드, 저장 및 파일 실행
save_grip_file(dict_nan_unq, output_path, "dict_nan_unq")

# 사전에 대입~ bf_stwd 를 af_stwd로

In [ ]:
# dict_nan에서 사전단어 수정 후 stwd 적용
before = dict_nan_unq['bf_stwd_kor']
after = dict_nan_unq['af_stwd_kor']

for i, j in zip(before, after):
    dict_nan.loc[dict_nan['stwd_kor'] == i, 'stwd_kor'] = j
dict_nan

# ISIN 점검

In [ ]:
# 사전 등록 점검(리스트 필요)
def dict_word_isin(data, c_list, col_name):
    # ''(공란) 제거
    data = data[data[col_name] != '']
    if (data[col_name].isin(c_list)==False).sum() == 0:
        print("모든 단어가 사전에 등록되어 있습니다.")

In [ ]:
dict_word_isin(dict_nan, dict_list, 'stwd_kor')

In [ ]:
dict_nan[(dict_nan['stwd_kor'].isin(dict_list)==False)]

In [ ]:
dict_nan['stwd_kor'].unique()

In [ ]:
dict_nan[dict_nan['stwd_kor'].isin(dict_list)]

In [ ]:
dict_list[dict_list == '가입']

In [ ]:
from varname import nameof

In [ ]:
nameof(stwd_total)

In [ ]:
def chk_mrg_idx(data1, data2, cp_data):
    rw_data = pd.concat([data1, data2]).sort_index()
    cnt = (cp_data.index == rw_data.index).sum()
    cnt_incs = (cp_data.index != rw_data.index).sum()
    # merge 전 index 목록과 다른것 비교
    if cnt_incs == 0:
        print("merge 전/후의 index가 같습니다. count:"+str(cnt))
        globals()["{}".format(nameof(stwd_total))] = rw_data
        print("수정 내용을 적용하였습니다 (변수명:"+ nameof(stwd_total) + ")")
    elif cnt_incs > 0:
        print("merge 전/후의 index가 다릅니다.count:"+str(cnt))


In [ ]:
chk_mrg_idx(dict_is, dict_nan, stwd_total)

In [ ]:
stwd_total

# 재로드~

In [ ]:
dict_nan_unq.to_excel(output_path + "dict_nan_unq.xlsx")

In [ ]:
dict_nan_unq = dict_nan['stwd_kor'].unique()
dict_nan_unq

In [ ]:
# 사전에 있는 단어 검색

stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘'])]

In [ ]:
a = pd.concat([
stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘']) == False],stwd_total[stwd_total['stwd_kor'].isin(dict_insp['어휘'])]
]).sort_index()
a

In [ ]:
(stwd_total['stwd_kor'] == a['stwd_kor']).sum()

In [ ]:
# 사전에 없는 단어 검색
def isin_dict(data, cpr_data, data_col, cpr_data)
    data[data[data_col].isin(cpr_data[cpr_data]) == False]

In [ ]:
path_dict = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
test = pd.read_csv(path_dict + '/dict_total/' + 'dict_expn_dup.csv', index_col=0)
test = test[test['어휘'].notna()]
print(test.shape)
test

In [ ]:
dict_insp_oneMore.loc[dict_insp_oneMore['어휘'] == '개설']

In [ ]:
test.loc[test['어휘'] == '개설']

In [ ]:
stwd_total[stwd_total['stwd_kor'].isin(test['어휘'])]

In [ ]:
stwd_total[stwd_total['stwd_kor'].isin(dict_insp_list)==False]

In [ ]:
for i in tqdm(dict_insp_list):
    print(i)

In [ ]:
stwd_total

In [ ]:
nll_to_blnk(stwd_total, 'all')

# 4. 공통 표준 우선 적용

In [ ]:
nll_to_blnk(stwd_total, 'all')

In [ ]:
def pub_stnd(data):
    if yes_or_no("영문 약어명을 공통 표준으로 통일 하시겠습니까?"):

        # 표준 파일 불러오기
        sheet_name = "공통표준단어"
        input_path = os.path.join(os.getcwd(), "input\\")
        public_stwd = pd.read_excel(input_path + "공공데이터 공통표준용어(2021.12월).xlsx"
                                    , sheet_name=sheet_name
                                    , header=0
                                    , index_col="NO")
        public_stwd = public_stwd[['공통표준단어명', '공통표준단어영문약어명']]

        # 표준 영문명 자동 기입(널 제외)

        # 표준 단어명 기준 left join (영문명이 null이면 비표준, notnull이면 표준)
        std_chk = data.merge(public_stwd, left_on = 'stwd_kor', right_on = '공통표준단어명', how = 'left')
        for i in range(std_chk.shape[0]):

            # joined DF의 영문명이 있을 경우 표준으로 대신 기입 
            notna_chk = std_chk['stwd_eng'].notna()[i]
            if notna_chk:
                std_chk.loc[i, 'stwd_eng'] = std_chk.loc[i, '공통표준단어영문약어명']

        std_chk_list = ['index', 'stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
        globals()["std_chk"] = std_chk[std_chk_list]

        print('한글명을 기준으로 공통(행안부) 표준을 우선 적용합니다.')
        print(std_chk.shape)
        print('OUT: std_chk')
#         print(std_chk.head())
    else:
        print('공통(행안부) 표준을 우선 적용하지 않고 표준화를 진행합니다.')    

In [ ]:
pub_stnd(stwd_total)
std_chk.tail()

In [ ]:
pub_stnd(row_blnk_stwd_kor)

In [ ]:
pub_stnd(stwd_total)
std_chk.tail()

In [ ]:
std_chk.to_excel(input_path + 'aa.xlsx')

In [ ]:
# if yes_or_no("영문 약어명을 공통 표준으로 통일 하시겠습니까?"):

#     # 표준 파일 불러오기
#     sheet_name = "공통표준단어"
#     input_path = os.path.join(os.getcwd(), "input\\")
#     public_stwd = pd.read_excel(input_path + "공공데이터 공통표준용어(2021.12월).xlsx", sheet_name=sheet_name, header=0)
#     public_stwd = public_stwd[['NO', '공통표준단어명', '공통표준단어영문약어명']]

#     # 표준 영문명 자동 기입(널 제외)

#     # 표준 단어명 기준 left join (영문명이 null이면 비표준, notnull이면 표준)
#     std_chk = stwd_total.merge(public_stwd, left_on = 'stwd_kor', right_on = '공통표준단어명', how = 'left')
#     for i in range(std_chk.shape[0]):
        
#         # joined DF의 영문명이 있을 경우 표준으로 대신 기입 
#         notna_chk = std_chk['stwd_eng'].notna()[i]
#         if notna_chk:
#             std_chk.loc[i, 'stwd_eng'] = std_chk.loc[i, '공통표준단어영문약어명']

#     std_chk_list = ['index', 'stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
#     std_chk = std_chk[std_chk_list]
    
#     print('한글명을 기준으로 공통(행안부) 표준을 우선 적용합니다.')
# else:
#     print('공통(행안부) 표준을 우선 적용하지 않고 표준화를 진행합니다.')
    
# print(std_chk.shape)
# std_chk.tail()
# std_chk.head()

# 5. 한/영  중복제거 및 통합

In [ ]:
# 한글명 중복 검사: 수기 수정 후 최대 unique 값 카운트
def find_unique_eng(df):
    
    kor_input_cnt = sum(df_step_1['stwd_kor'].notna())
    kor_unique_cnt = len(set(df_step_1['stwd_kor']))
    eng_input_cnt = sum(df_step_1['stwd_eng'].notna())
    eng_unque_cnt = len(set(df_step_1['stwd_eng']))
    
    print("==중복값 비교==")
    # (한)미입력값 개수
    print('(한)     not null   개수: ' + str(kor_input_cnt))
    # (한)unique 값 개수
    print("(한)''포함 unique값 개수: " + str(kor_unique_cnt)) 
    # (영)미입력값 개수
    print('(영)     not null   개수: ' + str(eng_input_cnt))
    # (영)unique 값 개수
    print("(영)''포함 unique값 개수: " + str(eng_unque_cnt)) 
    
    if (kor_input_cnt == eng_input_cnt) and (kor_unique_cnt == eng_unque_cnt):
        print('"한글명과 영문명의 unique 개수가 같습니다."')
    else:
        print('"한글명과 영문명의 unique 개수가 다릅니다."')

In [ ]:
def chk_unique(data):
    global df_step_1   
    df_step_1 = data
    # 중복제거 후 다시 합쳐서 검사(1안)

    # 한글 중복제거
    col_stwd_kor = ['stwd_kor', 'num_kor']
    unique_kor = df_step_1['stwd_kor'].drop_duplicates()
    stwd_kor_unique = df_step_1.loc[unique_kor.index][col_stwd_kor]
    print("한글명중복제거:"+ str(stwd_kor_unique.shape))
    # stwd_kor_unique

    # 영문 중복제거
    col_stwd_eng = ['stwd_eng', 'num_eng']
    unique_eng = df_step_1['stwd_eng'].drop_duplicates()
    stwd_eng_unique = df_step_1.loc[unique_eng.index][col_stwd_eng]
    print("영문명중복제거:"+ str(stwd_eng_unique.shape))
    # stwd_eng_unique.tail(100)

    # 한글+영문 일렬 세우기
    df_step_1 = pd.DataFrame(stwd_total['index'])
    df_step_1 = df_step_1.merge(
        stwd_kor_unique,
        left_index = True,
        right_index = True,
        how = 'left'
    ).merge(
        stwd_eng_unique,
        left_index = True,
        right_index = True,
        how = 'left'
    )
    print(df_step_1.shape)
    df_step_1

    df_step_1.to_excel(input_path + "df_step_1" + str(dt.datetime.now().strftime('%y%m%d_%H%M%S')) + ".xlsx")
    print(df_step_1.shape)
    df_step_1
    print("OUT: df_step_1")
    
chk_unique(std_chk)
find_unique_eng(df_step_1)
df_step_1

In [ ]:
nll_to_blnk(df_step_1, 'all')
df_step_1

In [ ]:
blnk_row_rmv(df_step_1, 'col', 'stwd_kor')
blnk_rmv_stwd_kor.tail()

In [ ]:
blnk_rmv_stwd_kor.to_excel(output_path + "blnk_rmv_stwd_kor.xlsx")

In [ ]:
df_step_1

In [ ]:
data

In [ ]:
data.iloc[i]

In [ ]:
data = df_step_1
data_cnt = len(data)
notna_list = []
for i in range(data_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if data.iloc[i].notna().sum() > 1:
        notna_list.append(i)
        notna_list

In [ ]:
row_nll_rmv(df_step_1, 'df_step_1')

In [ ]:
df_step_1

# * 한글/영문 단어수 매칭

## 6. 자동수정, 파일 업로드 점검

### 6.1.수기 수정값 표시(! y/n)

In [ ]:
### 값이 한개라도 있을 경우만 카운트 ###

df_step_1_cnt = len(df_step_1)
notna_list = []
for i in range(df_step_1_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if df_step_1.iloc[i].notna().sum() > 1:
        notna_list.append(i)

df_step_1 = df_step_1.loc[notna_list]
print(df_step_1.shape)

# 원래 표준과 병합(수기 수정을 위해)
df_step_1 = df_step_1.merge(
    input_test,
    left_on = 'index',
    right_on = 'No',
    how = 'left'
)

# 영문 개수 세기
dup_list_eng = []
dup_cnt_eng = []
for i in range(df_step_1.shape[0]):
    dup_list_eng = df_step_1['stwd_eng'].to_list().count(df_step_1.loc[i, 'stwd_eng'])
    dup_cnt_eng.append(dup_list_eng)
df_step_1 = pd.concat([df_step_1, pd.DataFrame(dup_cnt_eng, columns = {"eng_cnt"})], axis = 1)

# 컬럼 추리기
df_step_1_list = ['index', 'stwd_eng', 'stwd_kor', '영문약어명', '용어명 분리', 'eng_cnt']
df_step_1 = df_step_1[df_step_1_list]

# 비교군 저장
df_step_1_chk = df_step_1.copy()
df_step_1.head()

# 2차
# 널 제외 한글단어를 표준화된 풀데이터(df_step_1)를 다시검색하여 영문명 자동 기입 : 없는 것은 수동기입 필요 

# 영문 null 값 '' 채우기
df_step_1['stwd_eng'] = df_step_1['stwd_eng'].fillna("")

df_step_1['eng_cor_yn'] = ""
for i in range(df_step_1.shape[0]):
    blnk_chk = df_step_1.loc[i, 'stwd_eng'] == ''
    if blnk_chk :
        df_step_1.loc[i, 'stwd_eng'] = '!' + '_'.join(set(stwd_total.loc[(stwd_total['stwd_kor'] == df_step_1.loc[i, 'stwd_kor']), 'stwd_eng'].to_list()))
        df_step_1.loc[i, 'eng_cor_yn'] = 'n'

df_step_1_chk = df_step_1.copy()
        
# 순서
df_step_1_xlsx = df_step_1.sort_values(['stwd_eng'])
df_step_1_xlsx = df_step_1_xlsx[df_step_1_xlsx['eng_cor_yn'] == 'n']
df_step_1_xlsx.to_excel(output_path+"chk_eng" + ".xlsx")

# # 파일 실행 하지 않음
# os.system(output_path + "chk_eng.xlsx")

print(df_step_1_xlsx.shape)
df_step_1_xlsx.head()

### 6.2. 수정 후 재업로드(한글): 개발 미정

### 6.3. 수정 후 재업로드(영문)

#### 6.3. 영문수정 > re_load 만 탐색할 수 있도록 코드 간략화 필요(지금은 시간상 그냥 개발)

#### 6.3.1. 자동수정 후 재업로드(영문)

In [ ]:
# 최초 1회 수정 후 재업로드

# 수기 수정된 데이터
######## 가짜, 추후(chk_eng)로 수정 ########
re_load = pd.read_excel(output_path + 'chk_eng.xlsx', index_col='Unnamed: 0')

# 주소값 덮어쓰지 않게 따로 복사하기
df_step_1_cor = df_step_1.copy()

# 모두null일 경우(=전부기입된경우), 그냥 진행, 그렇지 않을 경우 수정된 값 적용 후 진행ㅇ
# Index 병합, 수기수정 대입, 그 외 null 지정
if re_load['stwd_eng'].isna().all():
    print("검수 통과")
else: 
    df_step_1_cor['stwd_eng'] =  re_load['stwd_eng']
    print("수기 수정 다시")
    
# 나머지 null 값은 본래 데이터(df_step_1) 값으로 채워넣기
for i in range(df_step_1_cor.shape[0]):
    if pd.isna(df_step_1_cor.loc[i, 'stwd_eng']):
        df_step_1_cor.loc[i, 'stwd_eng'] = df_step_1.loc[i, 'stwd_eng']
        df_step_1_cor.loc[i, 'eng_cor_yn'] = 'y'

    if df_step_1_cor.loc[i, 'eng_cor_yn'] == 'n':
        if df_step_1_cor.loc[i, 'stwd_eng'] != df_step_1_chk.loc[i, 'stwd_eng']:
            df_step_1_cor.loc[i, 'eng_cor_yn'] = 'y'
        elif df_step_1_cor.loc[i, 'stwd_eng'] == df_step_1_chk.loc[i, 'stwd_eng']:
            df_step_1_cor.loc[i, 'eng_cor_yn'] = 'n'

# 특수문자, 빈칸, null 점검
tmp_list = []
for i in range(df_step_1_cor.shape[0]):
    if len(re.findall('[^A-Z0-9+]', df_step_1_cor.loc[i, 'stwd_eng'])):
        tmp_list.append(i)
    elif df_step_1_cor.loc[i, 'stwd_eng'] == '':
        tmp_list.append(i)
    elif pd.isna(df_step_1_cor.loc[i, 'stwd_eng']):
        tmp_list.append(i)

# 특수문자 'n'으로
df_step_1_cor.loc[tmp_list, 'eng_cor_yn'] = 'n'

# 전체 파일 저장
df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")

# 영문 점검 파일 저장    
chk_eng = df_step_1_cor.loc[df_step_1_cor['eng_cor_yn'] == 'n']
chk_eng.to_excel(output_path + "chk_eng.xlsx")

# 파일 실행
os.system(output_path + "chk_eng.xlsx")

print(df_step_1_cor.shape)
df_step_1_cor.head()

#### 6.3.2. 수기수정 후 재업로드(영문)

In [ ]:
# 주소값 덮어쓰지 않게 따로 복사하기
df_step_1_cor_tmp = df_step_1_cor.copy()
# 2회 이상부터

def repeat_eng():
    global df_step_1_cor_tmp
    global chk_eng
    global tmp_list_eng
#     global re_load

    # 수기 수정 후 다시 업로드: na_filter=False
    re_load = pd.read_excel(output_path + 'chk_eng.xlsx', index_col='Unnamed: 0', na_filter=False)
    
    # Index 병합, 수기수정 대입, 그 외 ''null'' 지정
    for i in re_load.index:
        df_step_1_cor_tmp.loc[i, 'stwd_eng'] =  re_load.loc[i, 'stwd_eng']
    
    # 나머지 null 값은 본래 데이터(df_step_1) 값으로 채워넣기
    for i in range(df_step_1_cor_tmp.shape[0]):
        if df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] == 'n':
            if df_step_1_cor_tmp.loc[i, 'stwd_eng'] != df_step_1_chk.loc[i, 'stwd_eng']:
                df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'y'
            elif df_step_1_cor_tmp.loc[i, 'stwd_eng'] == df_step_1_chk.loc[i, 'stwd_eng']:
                df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'n'
                
        if pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_eng']):
            df_step_1_cor_tmp.loc[i, 'stwd_eng'] = df_step_1_cor.loc[i, 'stwd_eng']
            df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'y'

    # 특수문자, 빈칸, 소문자, 한글, null 점검
    tmp_list_eng = []
    for i in range(df_step_1_cor_tmp.shape[0]):
        if len(re.findall('[^A-Z0-9+]', df_step_1_cor_tmp.loc[i, 'stwd_eng'])):
            tmp_list_eng.append(i)
        elif df_step_1_cor_tmp.loc[i, 'stwd_eng'] == '':
            tmp_list_eng.append(i)
        elif pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_eng']):
            tmp_list_eng.append(i)

    # 특수문자 'n'으로
    df_step_1_cor_tmp.loc[tmp_list_eng, 'eng_cor_yn'] = 'n'

    # 파일 실행
    os.system(output_path + "chk_eng.xlsx")

    # 파일 저장
    chk_eng = df_step_1_cor_tmp.loc[df_step_1_cor_tmp['eng_cor_yn'] == 'n']
    df_step_1_cor_tmp.to_excel(output_path + "df_step_1_cor.xlsx")
    chk_eng.to_excel(output_path + "chk_eng.xlsx")

#### 6.3.3. 재업로드 반복 실행

In [ ]:
# 한글명, 영문명이 모두 ''값일 경우, 수정 완료
if (chk_eng[['stwd_kor', 'stwd_eng']] == '').all().all():
    print('"영문 수정 완료"' + " (빈칸 수:" + str(sum(df_step_1_cor_tmp['stwd_eng'] == '')) + ")")
    repeat_eng()
    os.system(output_path + "df_step_1_cor.xlsx")
else:
    if yes_or_no("영문명 수정이 필요합니다. 반복 점검하시겠습니까?"):
        repeat_eng()
    else:
        os.system(output_path + "chk_eng.xlsx")    

In [ ]:
# 함수화 (추후에)

# def repeat_eng():
#     global df_step_1_cor_tmp
#     global chk_eng
#     global tmp_list_eng

In [ ]:
# #다시 검사(한글 1회차)

# df_step_1_cor = df_step_1_cor_tmp.copy()

# df_step_1_cor['kor_cor_yn'] = 'n'

# # 한글 검사
# tmp_list=[]
# for i in range(df_step_1_cor.shape[0]):
#     # 영어가 들어갔따
#     if (len(re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1):
#         tmp_list.append(i)
#         print(i)
# che_kor = df_step_1_cor.loc[tmp_list]
# che_kor.to_excel(output_path + "chk_kor.xlsx")

# # 수기 수정 후 다시 업로드: na_filter=False
# re_load = pd.read_excel(output_path + 'chk_kor.xlsx', index_col='Unnamed: 0', na_filter=False)

# os.system(output_path + "chk_kor.xlsx")

# re_load

In [ ]:
#다시 검사(한글 1회차)

df_step_1_cor = df_step_1_cor_tmp.copy()

# kor_cor_yn 초기값 'n' 설정
df_step_1_cor['kor_cor_yn'] = ''

# 한글 검사
tmp_list=[]

for i in range(df_step_1_cor.shape[0]):
    cdtn_eng = len(re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1 # 영문이 한자라도 있을 시
    cdtn_kor = len(re.findall('[가-힣+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1 # 한글이 한자라도 있을 시
    cdtn_eng_lower = len(re.findall('[a-z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
    # 한글과 영문이 섞인 경우만: 대문자만 들어갈 경우 허용
    if cdtn_eng & cdtn_kor:
        tmp_list.append(i)
        print(i)
    # 소문자가 들어간 경우만
    elif cdtn_eng_lower:
        tmp_list.append(i)
        print(i)

df_step_1_cor.loc[tmp_list, 'kor_cor_yn'] = 'n'
chk_kor = df_step_1_cor.loc[tmp_list]
chk_kor.to_excel(output_path + "chk_kor.xlsx")

# 수기 수정 후 다시 업로드: na_filter=False
re_load = pd.read_excel(output_path + 'chk_kor.xlsx', index_col='Unnamed: 0', na_filter=False)

os.system(output_path + "chk_kor.xlsx")

re_load

In [ ]:
#     # 한글 검사
#     tmp_list=[]
#     for i in range(df_step_1_cor.shape[0]):
#         cdtn_eng = len(re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
#         cdtn_kor = len(re.findall('[가-힣+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
#         cdtn_eng_lower = len(re.findall('[a-z+]', df_step_1_cor.loc[i, 'stwd_kor'])) >= 1
#         # 영문 대문자만 들어간 경우 허용
#         if cdtn_eng:
#             tmp_list.append(i)
#             print(i)
#         # 한글 단어만 들어간 경우 허용
#         elif cdtn_kor:
#             tmp_list.append(i)
#             print(i)
#     # 올바르게 적용된 내용 'y'로
#     df_step_1_cor_tmp.loc[tmp_list_kor, 'kor_cor_yn'] = 'y'

# 다시다시... y n 바꾸는거 안됨..

In [ ]:
#다시 검사(한글 n회차)

def repeat_kor():
    global df_step_1_cor
    global chk_kor
    global tmp_list_kor

    # 수기 수정 후 다시 업로드: na_filter=False
    re_load = pd.read_excel(output_path + 'chk_kor.xlsx', index_col='Unnamed: 0', na_filter=False)
    re_load

    # 엑셀 수정내용 적용
    for i in range(df_step_1_cor.shape[0]):
        for j in re_load.index:
            if df_step_1_cor.index.to_list()[i] == j:
                df_step_1_cor.loc[i, 'stwd_kor'] = re_load.loc[j, 'stwd_kor']

    # 비교하기위한 복사
    df_step_1_cor_tmp = df_step_1_cor.copy()

    # 그외 null 값은 이전 데이터(df_step_1_cor) 값으로 채워넣기
    for i in range(df_step_1_cor_tmp.shape[0]):
        if df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] == 'n':
            # 이전 데이터와 비교하여 바뀐값 'y' 입력
            if df_step_1_cor_tmp.loc[i, 'stwd_kor'] != df_step_1_cor.loc[i, 'stwd_kor']:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'y'
            # 이전 데이터와 비교하여 바뀌지 않은 값 'n' 입력
            elif df_step_1_cor_tmp.loc[i, 'stwd_kor'] == df_step_1_cor.loc[i, 'stwd_kor']:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'n'
        
        # null일경우 '' 입력
        if pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_kor']):
            df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = ''

    # 특수문자, 빈칸, 소문자, 한글, null 점검
    tmp_list_kor = []
    for i in range(df_step_1_cor_tmp.shape[0]):
        
        # 수정필요(n)사항 중에서..
        if df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] == 'n':
            cdtn_eng_len = re.findall('[a-zA-Z+]', df_step_1_cor.loc[i, 'stwd_kor'])
            cdtn_kor_len = re.findall('[가-힣+]', df_step_1_cor.loc[i, 'stwd_kor'])
            # 한글만 한자 이상
            if cdtn_eng == 0 & cdtn_kor >= 1:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'y'
                print(i)
            # 영문만 한자 이상
            elif cdtn_eng >= 1 & cdtn_kor == 0:
                df_step_1_cor_tmp.loc[i, 'kor_cor_yn'] = 'y'
                print(i)
            
        # 소문자가 들어간 경우만
        elif cdtn_eng_lower:
            tmp_list_kor.append(i)
            print(i)
            
        # 허용값이 아닌 개수 1개 이상
        elif len(re.findall('[^가-힣A-z0-9+]', df_step_1_cor_tmp.loc[i, 'stwd_kor'])): 
            tmp_list_kor.append(i)
        elif df_step_1_cor_tmp.loc[i, 'stwd_kor'] == '':
            tmp_list_kor.append(i)
        elif pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_kor']):
            tmp_list_kor.append(i)

    # 특수문자 'n'으로
    df_step_1_cor_tmp.loc[tmp_list_kor, 'kor_cor_yn'] = 'n'
        
    chk_kor = df_step_1_cor_tmp.loc[df_step_1_cor_tmp['kor_cor_yn'] == 'n']
    chk_kor.to_excel(output_path + "chk_kor.xlsx")
    
    df_step_1_cor = df_step_1_cor_tmp
    df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")
    
    os.system(output_path + "chk_kor.xlsx")

In [ ]:
repeat_kor()

In [ ]:
df_step_1_cor

In [ ]:
df_step_1_cor_tmp

In [ ]:
# 동작 안됨
# 한글명, 영문명이 모두 ''값일 경우, 수정 완료
if (chk_kor[['stwd_kor', 'stwd_eng']] == '').all().all():
    print('"한글 수정 완료"' + " (빈칸 수:" + str(sum(df_step_1_cor['stwd_kor'] == '')) + ")")
    repeat_kor()
    os.system(output_path + "df_step_1_cor.xlsx")
else:
    if yes_or_no("한글명 수정이 필요합니다. 반복 점검하시겠습니까?"):
        repeat_kor()
    else:
        os.system(output_path + "chk_kor.xlsx")    

In [ ]:
df_step_1_cor_tmp

In [ ]:
# 영문 중복 검사
if sum(df_step_1_cor['stwd_eng'].duplicated(keep=False)) > 1: # keep=False: 중복중 하나만 유지하지 않고 중복인 경우 모두 체크한다
    print("영문명 중복을 탐지하였습니다.")
    
    
    #코드 작성
    
# 한글 중복 검사
elif sum(df_step_1_cor['stwd_kor'].duplicated(keep=False)) > 1:
    print("한글명 중복을 탐지하였습니다.")
    
    
    #코드 작성
    
else:
    print("단어 중복을 모두 확인하였습니다.")
    
#     #코드 작성(적용)
#     df_step_1_cor = df_step_1_cor_tmp
#     df_step_1_cor.to_excel(output_path +"df_step_1_cor.xlsx")
    
#     print("무의미한 값('')을 삭제하였습니다.")
#     df_space = df_step_1_cor_dup.loc[(df_step_1_cor_dup['stwd_eng'] == '') | (df_step_1_cor_dup['stwd_kor'] == '')]
#     n_space_list = list(set(df_step_1_cor_dup.index) - set(df_space.index))
#     df_step_1_cor = df_step_1_cor.loc[n_space_list]
#     df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")
    
#     print("단어 사전을 완성하였습니다.")
    os.system(output_path + "df_step_1_cor.xlsx")

In [ ]:
df_step_1_cor_kor_dup = df_step_1_cor[df_step_1_cor['stwd_kor'].duplicated(keep=False)]
df_step_1_cor_kor_dup.to_excel(output_path + "df_step_1_cor_kor_dup.xlsx")
df_step_1_cor_kor_dup

In [ ]:

# 영문 중복 검사
if sum(df_step_1_cor['stwd_eng'].duplicated(keep=False)) > 1: # keep=False: 중복중 하나만 유지하지 않고 중복인 경우 모두 체크한다
    print("영문명 중복을 탐지하였습니다.")
    
    
    #코드 작성
    
# 한글 중복 검사
elif sum(df_step_1_cor['stwd_kor'].duplicated(keep=False)) > 1:
    print("한글명 중복을 탐지하였습니다.")
    
    
    
    #코드 작성
    
else:
    
    
    print("단어 중복을 모두 확인하였습니다.")
    print("무의미한 값('')을 삭제하였습니다.")
    df_space = df_step_1_cor.loc[(df_step_1_cor['stwd_eng'] == '') | (df_step_1_cor['stwd_kor'] == '')]
    n_space_list = list(set(df_step_1_cor.index) - set(df_space.index))
    df_step_1_cor = df_step_1_cor.loc[n_space_list]
    df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")
    print("단어 사전을 완성하였습니다.")
    os.system(output_path + "df_step_1_cor.xlsx")

In [ ]:
df_step_1_cor_dup = df_step_1_cor[df_step_1_cor['stwd_kor'].duplicated(keep=False)]
df_step_1_cor_dup.to_excel(output_path + "df_step_1_cor_dup.xlsx")
df_step_1_cor_dup

In [ ]:
df_step_1_cor_dup = pd.read_excel(output_path + "df_step_1_cor_dup.xlsx")
# df_step_1_cor_dup.to_excel(output_path + "df_step_1_cor_dup.xlsx")
df_step_1_cor_dup

In [ ]:
df_step_1_cor_dup['stwd_kor'].duplicated(keep=False)

In [ ]:
# 엑셀 수정내용 적용
for i in range(df_step_1_cor.shape[0]):
    for j in df_step_1_cor_dup.index:
        if df_step_1_cor.index.to_list()[i] == j:
            df_step_1_cor.loc[i, 'stwd_kor'] = df_step_1_cor_dup.loc[j, 'stwd_kor']

In [ ]:
df_step_1_cor

In [ ]:
df_step_1_cor_dup.index

In [ ]:
sum(df_step_1_cor['stwd_eng'].duplicated(keep=False)) > 1

In [ ]:
sum(df_step_1_cor['stwd_kor'].duplicated(keep=False)) > 1

In [ ]:
df_step_1_cor.loc[df_step_1_cor['stwd_kor'].duplicated(keep=False)]

In [ ]:
[단어]
[v]한글명이상 검사 (예: 영문 대문자)
[v]한글 하나,영문 하나


[]다시, 한글 영어 중복검사
[]단어 순서 검사

[]스페이스 제거하여 완전 사전?

<용어>
[]단어 개수 검사 (가A ID 생성 후 일치화)



In [ ]:
os.system(output_path + "df_step_1_cor.xlsx")

In [ ]:
stwd_kor

In [ ]:
            
        # 한국어 점검
        if len(re.findall('[^A-Z0-9가-힣+]', df_step_1_cor_tmp.loc[i, 'stwd_kor'])):
            tmp_list_eng.append(i)
        elif df_step_1_cor_tmp.loc[i, 'stwd_kor'] == '':
            tmp_list_eng.append(i)
        elif pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_kor']):
            tmp_list_eng.append(i)

In [ ]:
def eng_blnk_chk():
    global chk_eng
    global df_step_1_cor_tmp
    
    # count 점검: 빈칸은 한개만 허용함
    cnt_chk = sum(df_step_1_cor_tmp.loc[tmp_list, 'stwd_eng'] == '')
    if cnt_chk == 1:
        print("빈칸 허용치와 일치: " + str(cnt_chk))
        if sum(chk_eng.loc[chk_eng['eng_cor_yn'] == 'n', 'stwd_eng'] !=  '') == 0:
            print('"영문 수정 완료"')
        else:
            print('추가 수정 필요')
    elif cnt_chk < 1:
        print("빈칸이 허용치보다 적음: " + str(cnt_chk))
    elif cnt_chk >= 2:
        print("빈칸이 허용치보다 많음: " + str(cnt_chk))

In [ ]:
여기에 의존하면 안돼..

In [ ]:
# '' == 1개, 'n' 중에서 ''가 하나도 없을 경우 작업 종료 
condition_1 = sum(df_step_1_cor_tmp.loc[tmp_list, 'stwd_eng'] == '') == 1
# condition_2 = sum(chk_eng.loc[chk_eng['eng_cor_yn'] == 'n', 'stwd_eng'] !=  '') == 0
condition_2 = sum(chk_eng.loc[chk_eng['eng_cor_yn'] == 'n', 'stwd_eng'] == '') == 1
if condition_1 and condition_2:
    print("더 이상 진행할 수 없습니다.")
    os.system(output_path + "chk_eng.xlsx")
    os.system(output_path + "df_step_1_cor.xlsx")
else:
    if yes_or_no("반복 점검하시겠습니까?"):
        repeat_eng()
        eng_blnk_chk()
    else:
        os.system(output_path + "chk_eng.xlsx")

In [ ]:
df_step_1 = df_step_1_cor_tmp[df_step_1_cor_tmp['stwd_eng'].notna()]
df_step_1 = df_step_1_cor_tmp[df_step_1_cor_tmp['stwd_eng'] != '']

In [ ]:
# count 확인
find_unique_eng(df_step_1)

In [ ]:
# count 확인

if sum(df_step_1_cor_tmp['stwd_eng'].str.contains('!', '_')) == 0:
    print('영문명을 모두 수정하였습니다 !_수: '+ str(sum(df_step_1_cor_tmp['stwd_eng'].str.contains('!', '_'))))
    print(df_step_1_cor_tmp.shape)
else:
    print('아직 수정하지 않은 영문명이 남아있습니다 !_수: '+ str(sum(df_step_1_cor_tmp['stwd_eng'].str.contains('!', '_'))))    
    print(df_step_1_cor_tmp.shape)
    
# 두 카운트가 같아아 최종 표준화 점검 완료 1:1 대응
if len(set(df_step_1['stwd_kor'])) == len(set(df_step_1['stwd_eng'])):
    print('영문명과 한글명의 고유개수가 일치합니다')
print("영문명: " + str(len(set(df_step_1['stwd_eng']))))
print("한글명: " + str(len(set(df_step_1['stwd_kor']))))

In [ ]:
print('_'.join(stwd_total.loc[stwd_total['index'] == 1055, 'stwd_kor'].to_list()))
stwd_total.loc[stwd_total['index'] == 1055, 'stwd_kor']

In [ ]:
df_step_1['stwd_kor'].isna()[i]

In [ ]:
df_step_1.to_excel(output_path+"chk_eng" + ".xlsx")

In [ ]:
len(set(df_step_1['stwd_eng']))

In [ ]:
df_step_1

In [ ]:
index = [1055, 1056, 1057, 445, 432, 724, 878, 959, 1011, 934, 781, 454, 239]

In [ ]:
stwd_total.loc[(stwd_total['stwd_eng'] == 'NUM') & (stwd_total['stwd_kor'] != ''), 'stwd_kor'].values[0]

In [ ]:
df_step_1.loc[(df_step_1['stwd_eng'] == 'NUM') & (df_step_1['stwd_kor'] != ''), 'stwd_kor'].values[0]

In [ ]:
# 널 제외 한글단어를 표준화된 풀데이터(df_step_1)를 다시검색하여 영문명 자동 기입 : 없는 것은 수동기입 필요
for i in range(tmp_df_step_1.shape[0]):
    isna_chk = tmp_df_step_1['stwd_kor'].isna()[i]
    if isna_chk:
#         tmp_df_step_1.loc[i, 'stwd_kor'] = stwd_total.loc[(stwd_total['stwd_eng'] == 'NUM') & (stwd_total['stwd_kor'] != ''), 'stwd_kor'].values[0]
        tmp_df_step_1.loc[i, 'stwd_kor'] = df_step_1.loc[(df_step_1['stwd_eng'] == 'NUM') & (df_step_1['stwd_kor'] != ''), 'stwd_kor'].values[0]

    
# df_step_1_list = ['index', 'stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
# tmp_df_step_1 = tmp_df_step_1[df_step_1_list]
# print(tmp_df_step_1.shape)
# tmp_df_step_1.head()
std_chk_eng

# 순서
tmp_df_step_1 = tmp_df_step_1.sort_values(['stwd_eng'])
tmp_df_step_1.to_excel(output_path+"chk_eng" + ".xlsx")

print(tmp_df_step_1.shape)
tmp_df_step_1

In [ ]:
null_cnt = 0
if null_cnt > 0:
    print("빈칸이 있습니다")
### ㅅ수정 후 재업로드 하여 null count
# ''개수가 1 이상일 경우 일경우 되묻지 않고 안내만
# ''개수가 0일경우 되묻지 않고 안내만


In [ ]:
stwd_total.loc[stwd_total['index'] == 781, 'stwd_kor'].values

In [ ]:
index = [1055, 1056, 1057, 445, 432, 724, 878, 959, 1011, 934, 781, 454, 239]
for i in range(len(index)):
    a = stwd_total.loc[stwd_total['index'] == index[i], 'stwd_kor']
    print(a)
    
a

In [ ]:
df_step_1.loc[(df_step_1['stwd_eng'] == 'NUM') & (df_step_1['stwd_kor'] != ''), 'stwd_kor'].values[0]

In [ ]:
stwd_total.loc[stwd_total['stwd_eng'] == 'HAKJUM', 'stwd_kor'].values[0]

In [ ]:
# 한국어가 null값이면 자동 영문명 자동 기입


In [ ]:
tmp_df_step_1

In [ ]:
tmp_df_step_1

In [ ]:







print(df_step_1.shape)
df_step_1.head()

In [ ]:
std_chk

In [ ]:
std_chk

In [ ]:



a.to_excel(output_path + "dfsfd.xlsx")
# dup_cnt_eng

#     dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
#     globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)

        
# for i in range(eng_firts_cnt):
#     globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
#     globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]

In [ ]:
df_step_1['stwd_eng'].to_list().count(df_step_1.loc[0, 'stwd_eng'])

In [ ]:
dupl_list_eng0['dupl_list_eng0'].to_list().count(1)

In [ ]:
tmp_df_step_1.shape[0]

In [ ]:
# 중복 지우기 위해


In [ ]:
# 널 채우기 위해
tmp_df_step_1[stwd_kor]

In [ ]:
df_step_1.shape

In [ ]:
df_step_1.to_excel(input_path+"111.xlsx")
std_chk.to_excel(input_path+"112.xlsx")

In [ ]:
# public_stwd_cnt =public_stwd.shape[0]
# df_step_1_cnt = df_step_1.shape[0]

# for i in range(df_step_1_cnt):
#     for j in range(public_stwd_cnt):
#         if df_step_1.iloc[i,1] == public_stwd.iloc[j,1]: #[i, 0]
#             # 영문명 수정
#             df_step_1.iloc[i,3] = public_stwd.iloc[j,2]
#         else:
#             a = 1
# #             df_step_1.iloc[i,1] = "그대로"+df_step_1.iloc[i,1]

In [ ]:
# 파일 저장 묻기
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')    

if yes_or_no('파일을 저장하시겠습니까?'):
    file_name = 'stwd_total_manu1' + "_" + str(dt.datetime.now().strftime('%y%m%d_%H%M%S') + ".xlsx")
    print('[엑셀 파일을 생성하였습니다]: ' + file_name )
else:
    print('[검토한 후 다시 업로드 해주세요]')
stwd_total.head()

# 중복제거 로직

In [ ]:
## 중복 제거 > 
1. 한개라도 값이 있는 경우 모두 추가
2. 영문 unique 기준 영문 중복 검정 (이미 한글 중복검정 돼 있음: 표준데이터와 병합하기 때문에..) 
/ 중복 여부 검사
3. 한글 unique 기준 한글 중복 검정 
/ 중복 여부 검사
4. ID 기준
- 수기수정 2부터~ n회 반복 / 중복 여부 검사 
=> 모든 중복 제거 , 빈값 넣기, 잘못된값 수정
5. 사전 최종 검사 및 업로드
- n회 반복 -



In [ ]:
# 중복제거 후 다시 합쳐서 검사(1안)

# 한글 중복제거
col_stwd_kor = ['stwd_kor', 'num_kor']
unique_kor = stwd_total['stwd_kor'].drop_duplicates()
stwd_kor_unique = stwd_total.loc[unique_kor.index][col_stwd_kor]
print("한글명중복제거:"+ str(stwd_kor_unique.shape))
# stwd_kor_unique

# 영문 중복제거
col_stwd_eng = ['stwd_eng', 'num_eng']
unique_eng = stwd_total['stwd_eng'].drop_duplicates()
stwd_eng_unique = stwd_total.loc[unique_eng.index][col_stwd_eng]
print("영문명중복제거:"+ str(stwd_eng_unique.shape))
# stwd_eng_unique.tail(100)

# 한글+영문 일렬 세우기
df_step_1 = pd.DataFrame(stwd_total['index'])
df_step_1 = df_step_1.merge(
    stwd_kor_unique,
    left_index = True,
    right_index = True,
    how = 'left'
).merge(
    stwd_eng_unique,
    left_index = True,
    right_index = True,
    how = 'left'
)
print(df_step_1.shape)
df_step_1

### 값이 한개라도 있을 경우 카운트 ###

total_cnt = len(df_step_1)
null_list = []
for i in range(total_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if df_step_1.iloc[i].notna().sum() > 1:
        null_list.append(i)

df_step_1 = df_step_1.loc[null_list]

df_step_1 = df_step_1.merge(
    input_test,
    left_on = 'index',
    right_on = 'No',
    how = 'left'
)

df_step_1.to_excel(input_path + "df_step_1" + str(dt.datetime.now().strftime('%y%m%d_%H%M%S')) + ".xlsx")
print(df_step_1.shape)
df_step_1

# 여기서부터 다시

In [ ]:
# 중복 제거 다시 묻기
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')    
chk_unique()
if yes_or_no('파일을 업로드하여 중복검사를 재실행 하시겠습니까?'):
    
    
    ## 엑셀 READ한 후 중복검사 다시!
#     sheet_name = "수정이력_220925"
    df_step_1 = pd.read_excel(input_path, sheet_name=sheet_name, header=0)
    df_step_1
    
    file_name = 'df_step_1_re' + "_" + str(dt.datetime.now().strftime('%y%m%d_%H%M%S') + ".xlsx")
    
    chk_unique()
    
    df_step_1.to_excel(df_step_1, input_path + "df_step_1" + ".xlsx")
    print('[엑셀 파일을 생성하였습니다]: ' + file_name )
else:
    print('[진행하겠습니다]')
# df_step_1_re.head()

In [ ]:
df_step_1.loc[24].notna()

In [ ]:
tmp_col = ['stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
notna_list = []
df_step_1 = df_step_1[tmp_col]
for i in range(len(df_step_1TT)):
    # 하나라도 값이 있다면 False, 모두 null일경우 True
    if df_step_1.loc[i].isna().all() == False:
        notna_list.append(i)
notna_list
df_step_1 = df_step_1.loc[notna_list]
print(df_step_1.shape)
df_step_1

In [ ]:
# df_step_1 = df_step_1[['index', 'stwd_kor', 'stwd_eng', 'num_kor', 'num_eng']]

# # unique 뽑기 & null 값 제거
df_step_uniq = df_step_1.dropna(axis = 0)
df_step_uniq
# df_step_uniq = df_step_uniq.sort_values(['stwd_kor', 'stwd_eng'])
# print(df_step_uniq.shape)

# df_step_uniq = df_step_uniq.merge(input_test, left_on='index', right_on='No', how='left')
# print(df_step_uniq.shape)

# df_step_uniq.tail(500)

In [ ]:
df_step_1.dropna(axis = 0).isna()

In [ ]:
df_step_1TT = df_step_1[]

In [ ]:
df_step_1.shape

In [ ]:
stwd_total_cnt

In [ ]:
df_step_1TT.loc[100].isna().all()

In [ ]:
df_step_1TT

In [ ]:
df_step_1TT

In [ ]:
len(notna_list)

In [ ]:
df_step_1TT.loc[13639].isna().all()

In [ ]:
df_step_1[df_step_1.notna()]

In [ ]:
# 중복제거 후 다시 합쳐서 검사(1안)

# 한글 중복제거
col_stwd_kor = ['stwd_kor', 'num_kor']
unique_kor = stwd_total['stwd_kor'].drop_duplicates()
stwd_kor_unique = stwd_total.loc[unique_kor.index][col_stwd_kor]
print("한글명중복제거:"+ str(stwd_kor_unique.shape))
# stwd_kor_unique

# 영문 중복제거
col_stwd_eng = ['stwd_eng', 'num_eng']
unique_eng = stwd_total['stwd_eng'].drop_duplicates()
stwd_eng_unique = stwd_total.loc[unique_eng.index][col_stwd_eng]
print("영문명중복제거:"+ str(stwd_eng_unique.shape))
# stwd_eng_unique.tail(100)

# 한글+영문 일렬 세우기
df_step_1 = pd.DataFrame(stwd_total['index'])
df_step_1 = df_step_1.merge(
    stwd_kor_unique,
    left_index = True,
    right_index = True,
    how = 'left'
).merge(
    stwd_eng_unique,
    left_index = True,
    right_index = True,
    how = 'left'
)
# df_step_1 = df_step_1[['index', 'stwd_kor', 'stwd_eng', 'num_kor', 'num_eng']]

# # unique 뽑기 & null 값 제거
df_step_uniq = df_step_1.dropna(axis = 0)
df_step_uniq
# df_step_uniq = df_step_uniq.sort_values(['stwd_kor', 'stwd_eng'])
# print(df_step_uniq.shape)

# df_step_uniq = df_step_uniq.merge(input_test, left_on='index', right_on='No', how='left')
# print(df_step_uniq.shape)

# df_step_uniq.tail(500)

In [ ]:
df_step_1.dropna(axis = 0)

In [ ]:
print(df_step_uniq.shape)
print(df_step_uniq.shape)
print(df_step_1.shape)

In [ ]:
df_step_uniq.to_excel(input_path + "220928.xlsx")

In [ ]:
## 파일 생성 -> 중복 없어질때까지 n회 반복

## step1.1. 끝
일렬 나열 및 단순 중복 제거

## step1.2. 시작
스텝 1.2. 는 중복제거 한 파일을 ID를 기준으로 다시 중복제거 한 후, 재점검하여, 아닌 것 같은 경우는 초기 input 파일(원본)처럼 만듦

In [ ]:
stwd_total

In [ ]:
kor_emti_chk()

In [ ]:
df_lower_kor 

In [ ]:
pd.to_numeric(df_step_raw1['id_uniq_chk{}'.format(i)])

In [ ]:
# stwd_kor_unique = stwd_total.iloc[stwd_kor_unique.index]
# stwd_eng_unique = stwd_total.iloc[stwd_eng_unique.index]
stwd_eng_unique

In [ ]:
stwd_total['stwd_eng'][7289]

In [ ]:
stwd_total['stwd_eng'].drop_duplicates()

In [ ]:
# id 유니크 체크
# id 컬럼 만들기
stwd_total['id'] = stwd_total['stwd_kor'] + stwd_total['stwd_eng']

# uniq 항목 체크하기
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
stwd_total['uniq_chk'] = ''
stwd_total.loc[stwd_unique_list.index, 'uniq_chk'] = 1
stwd_total

In [ ]:
# 컬럼수만큼 테이블 만들고, 유니크 체그(한글명 id로만)
df_step_raw1 = df_index
for i in range(col_cnt):
    globals()['stwd_kor_uinq{}'.format(i)] = stwd_total[stwd_total['num_kor'] == 'num_kor{}'.format(i)]
    kor_list = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor', 'uniq_chk', 'id']
    globals()['stwd_kor_uinq{}'.format(i)] = globals()['stwd_kor_uinq{}'.format(i)][kor_list]
    globals()['stwd_kor_uinq{}'.format(i)].columns = [
        'index', 
        'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i),
        'id_uniq_chk{}'.format(i),
        'id{}'.format(i)
    ]
    globals()['stwd_eng_uinq{}'.format(i)] = stwd_total[stwd_total['num_eng'] == 'num_eng{}'.format(i)]
    eng_list = ['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']
    globals()['stwd_eng_uinq{}'.format(i)] = globals()['stwd_eng_uinq{}'.format(i)][eng_list]
    globals()['stwd_eng_uinq{}'.format(i)].columns = [
        'index', 
        'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i)
    ]
    # 한글,영문 통합하여 원본데이터 형태로
    df_step_raw1 = df_step_raw1.merge(
        globals()['stwd_kor_uinq{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uinq{}'.format(i)], 
        on ='index',
        how = 'left'
    )
    
# 각 row의 unique(id) 반복 횟수 적기 
id_uniq_chk_sum = 0
for i in range(col_cnt):
    globals()['id_uniq_chk{}'.format(i)] = pd.to_numeric(df_step_raw1['id_uniq_chk{}'.format(i)])
    id_uniq_chk_sum = id_uniq_chk_sum + globals()['id_uniq_chk{}'.format(i)].fillna(0)
id_uniq_chk_sum = pd.DataFrame(id_uniq_chk_sum, columns = ['id_uniq_chk_sum'])

# 계산값 df에 추가하기
df_step_raw1 = df_step_raw1.merge(
    id_uniq_chk_sum,
    left_index = True,
    right_index = True,
    how = 'left'
)

df_step_raw1

In [ ]:
df_step_raw1[tmp_col_list]

In [ ]:
# null 제거
df_step_1 = df_step_raw1[df_step_raw1['stwd_kor0'].notna()]

# 빈칸 제거
df_step_1 = df_step_1.loc[df_step_1['stwd_kor0'] != ""]

# 필요 컬럼 추리기
tmp_col_list = ['index', 'id_uniq_chk_sum']
for i in range(col_cnt):
    tmp_col_list.append('stwd_kor{}'.format(i))
    tmp_col_list.append('stwd_eng{}'.format(i))
    tmp_col_list.append('id_uniq_chk{}'.format(i))
    tmp_col_list.append('id{}'.format(i))
df_step_1 = df_step_1[tmp_col_list]

# row의 id 반복 횟수 적기(1 이상) 
df_step_1 = df_step_1.loc[df_step_1['id_uniq_chk_sum'] >= 1]

# 순서 정리
df_step_1 = df_step_1.sort_values(['stwd_kor0', 'stwd_eng0'])

print('카운트_전 :'+ str(df_step_raw1.shape))
print('카운트_후 :'+ str(df_step_1.shape))
# df_step_1.to_excel(input_path + "1step안_완.xlsx")
df_step_1.to_excel(input_path + "1step안_완Real.xlsx")
df_step_1.head(500)

In [ ]:

# 계산값 df에 추가하기
df_step_1 = df_step_1.merge(
    id_uniq_chk_sum,
    left_index = True,
    right_index = True,
    how = 'left'
)
df_step_1

In [ ]:
# 그냥 다시 합치기(한/영)
df_exam_raw1 = df_index

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total[stwd_total['num_kor'] == 'num_kor{}'.format(i)]
    kor_list = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']
    globals()['stwd_kor_uni{}'.format(i)] = globals()['stwd_kor_uni{}'.format(i)][kor_list]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_total[stwd_total['num_eng'] == 'num_eng{}'.format(i)]
    eng_list = ['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']
    globals()['stwd_eng_uni{}'.format(i)] = globals()['stwd_eng_uni{}'.format(i)][eng_list]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )
    

df_exam_raw1

In [ ]:

#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

df_exam_1 = df_exam_raw1.sort_values(['index', 'stwd_kor0'])
print(df_exam_1.shape)
df_exam_raw1.tail(50)
# df_exam_raw1.head(500)

In [ ]:
stwd_unique_list.index

In [ ]:
# (일렬)id 유니크 만 탐색
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
a = stwd_total.iloc[stwd_unique_list.index]
a


In [ ]:
stwd_unique

In [ ]:
# 우선 그대로 원복하고, index list 만들어서 조회만..?
stwd_total['id'] = stwd_total['stwd_kor'] + stwd_total['stwd_eng']
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
print(stwd_unique.shape)
stwd_unique
stwd_kor_unique = stwd_unique[['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']]
stwd_eng_unique = stwd_unique[['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']]
stwd_kor_unique

In [ ]:
# 다시 합치기(한/영)
df_exam_raw1 = a

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_eng_unique[stwd_eng_unique['num_eng'] == 'num_eng{}'.format(i)]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

df_exam_1 = df_exam_raw1.sort_values(['index', 'stwd_kor0'])
print(df_exam_1.shape)
df_exam_raw1.tail(50)
# df_exam_raw1.head(500)

In [ ]:
a.to_excel(input_path + "3안.xlsx")

In [ ]:
# [1안] 중복제거(새로운 id 셍상)
stwd_total['id'] = stwd_total['stwd_kor'] + stwd_total['stwd_eng']
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
print(stwd_unique.shape)
stwd_unique
stwd_kor_unique = stwd_unique[['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']]
stwd_eng_unique = stwd_unique[['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']]
stwd_kor_unique

In [ ]:
# 다시 합치기(한/영)
df_exam_raw1 = df_index

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_eng_unique[stwd_eng_unique['num_eng'] == 'num_eng{}'.format(i)]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

df_exam_1 = df_exam_raw1.sort_values(['index', 'stwd_kor0'])
print(df_exam_1.shape)
df_exam_raw1.tail(50)
# df_exam_raw1.head(500)

In [ ]:
avg_avg_kor


# sdfsㄹ


In [ ]:
total_cnt = avg_avg_kor
null_list = []
for i in range(total_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if df_exam_1.iloc[i].notna().sum() > 1:
        null_list.append(i)
        
#         print(i)
df_exam_1 = df_exam_1.loc[null_list]
df_exam_1.to_excel(input_path + "1안.xlsx")
df_exam_1


In [ ]:
# df_id = pd.DataFrame(stwd_total['id'])
# df_exam_1 = df_exam_1.merge(df_id, left_index=True, right_index=True, how='left').tail()

In [ ]:
# # null 제거
# df_exam_1 = df_exam_raw1[df_exam_raw1['stwd_kor0'].notna()]

# # 빈칸 제거
# df_exam_1 = df_exam_1.loc[df_exam_1['stwd_kor0'] != ""]

# 필요 컬럼 추리기
tmp_col_list = ['index']
for i in range(col_cnt):
    tmp_col_list.append('stwd_kor{}'.format(i))
    tmp_col_list.append('stwd_eng{}'.format(i))
tmp_col_list

df_exam_1 = df_exam_1[tmp_col_list]
# df_exam_1 = pd.concat([df_exam_1, df_id] ,axis=1)
# id_unique_list = df_id['id'].drop_duplicates()
# df_exam_1 = df_exam_1.loc[id_unique_list.index]
print('2안과 카운트 비교:'+ str(df_exam_1.shape))
df_exam_1.to_excel(input_path + "1안.xlsx")
df_exam_1.head()



In [ ]:
re.sub('[A-Z0-9가-힣+]', '', stwd_total['stwd_kor'][i])

In [ ]:
# 2안: 각각 중복제거 후 다시 합쳐서 검사
# 한글 중복제거
col_stwd_kor = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']
unique_kor = stwd_total['stwd_kor'].drop_duplicates()
stwd_kor_unique = stwd_total.loc[unique_kor.index][col_stwd_kor]
print(stwd_kor_unique.shape)
stwd_kor_unique

# 참고

In [ ]:
# 2안: 각각 중복제거 후 다시 합쳐서 검사
# 한글 중복제거
col_stwd_kor = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']
unique_kor = stwd_total['stwd_kor'].drop_duplicates()
stwd_kor_unique = stwd_total.loc[unique_kor.index][col_stwd_kor]
print(stwd_kor_unique.shape)
stwd_kor_unique

# 영문 중복제거
col_stwd_eng = ['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']
unique_eng = stwd_total['stwd_eng'].drop_duplicates()
stwd_eng_unique = stwd_total.loc[unique_eng.index][col_stwd_eng]
print(stwd_eng_unique.shape)
stwd_eng_unique

# 다시 합치기(한/영)
df_exam_raw1 = df_index

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_eng_unique[stwd_eng_unique['num_eng'] == 'num_eng{}'.format(i)]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

print(df_exam_raw1.shape)
df_exam_raw1.head()

# null 제거
df_exam_1 = df_exam_1[df_exam_1.notna()]

# 빈칸 제거
df_exam_1 = df_exam_1.loc[df_exam_1['stwd_kor0'] != ""]

# 필요 컬럼 추리기
tmp_col_list = ['index']
for i in range(col_cnt):
    tmp_col_list.append('stwd_kor{}'.format(i))
    tmp_col_list.append('stwd_eng{}'.format(i))
tmp_col_list

df_exam_1 = df_exam_1[tmp_col_list]
print('1안과 카운트 비교:'+ str(df_exam_1.shape))
df_exam_1.to_excel(input_path + "2안.xlsx")
df_exam_1.head()

# 널값 카운트
e = std_chk.loc[4, 'stwd_eng']
# type(e)
type(e) and pd.isna(e)

pd.notna(e)

In [ ]:
## df_exam_1 와 원본 붙이기!!

In [ ]:
unique_kor

In [ ]:
df_exam_1.to_excel(input_path + "df_exam_1.xlsx")
df_exam_1.to_excel(input_path + "df_exam_1.xlsx")
stwd_total.to_excel(input_path + "stwd_total.xlsx")

In [ ]:
STOP!!!!!!!!!!

In [ ]:
stwd_total

In [ ]:
# 1차 검토
# 유니크(한 * 영) 데이터 뽑아 엑셀 생성
# 수정후 total 데이터 뽑아 엑셀 생성


In [ ]:
stwd_total_kor_unique

In [ ]:
stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]

In [ ]:
tt = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(0)]
tt['stwd_kor']
tt

In [ ]:
for i in range(kor_firts_cnt):
#     a = pd.DataFrame()
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    print(i)

In [ ]:
ba = df_index.merge(globals()['stwd_kor_uni{}'.format(0)], left_on = 'index', right_on = 'index', how = 'left')
ba

In [ ]:
# 다시 합치기
df_index_tmp = df_index

for i in range(col_cnt):
    a = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = ['index', 'stwd_kor{}'.format(i), 'num_kor{}'.format(i), 'dupl_list_kor{}'.format(i)]
#     tmp['num_kor{}'.format(i)] = a['stwd_kor']
    aa = a.merge(globals()['stwd_kor_uni{}'.format(i)], left_on = 'index', right_on = 'index', how = 'left')
    print('===========================================================================')
    print(a)
    df_index_tmp = df_index_tmp.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        left_on ='index',
        right_on = 'index',
        how = 'left'
    )


df_index_tmp 

In [ ]:
df_index_tmp

In [ ]:
df_index_tmp[df_index_tmp['stwd_kor{}'.format(0)].isna()]

In [ ]:
# null제거
for i in range(col_cnt):
    df_index_tmp['stwd_kor{}'.format(i)] = df_index_tmp['stwd_kor{}'.format(i)].fillna("")
    df_index_tmp['num_kor{}'.format(i)] = df_index_tmp['num_kor{}'.format(i)].fillna("")
    df_index_tmp['dupl_list_kor{}'.format(i)] = df_index_tmp['dupl_list_kor{}'.format(i)].fillna("")

In [ ]:
    
# null값 수정
for i in range(col_cnt):
    df_index_tmp['stwd_kor{}'.format(i)] = df_index_tmp['stwd_kor{}'.format(i)].fillna("")
    df_index_tmp['num_kor{}'.format(i)] = df_index_tmp['num_kor{}'.format(i)].fillna("")
    df_index_tmp['dupl_list_kor{}'.format(i)] = df_index_tmp['dupl_list_kor{}'.format(i)].fillna("")

In [ ]:
df_index_tmp.iloc[:,30]

In [ ]:
df_index_tmp['stwd_kor{}'.format(1)].fillna("")

In [ ]:

    
    
    
    
#     input_split_kor["num_kor{}".format(i)] = "num_kor{}".format(i)
#     globals()["stwd_kor{}".format(i)] = pd.concat(
#             [
#             df_index.fillna(""), 
#             pd.DataFrame(input_split_kor["stwd_kor{}".format(i)]).fillna(""),
#             pd.DataFrame(input_split_kor["num_kor{}".format(i)]).fillna(""),
#             globals()["dupl_list_kor{}".format(i)].fillna("")
#             ], 

In [ ]:
df_index_tmp = df_index
df_index_tmp 

In [ ]:
for i in range(kor_firts_cnt):
    a = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = ['index', 'stwd_kor{}'.format(i), 'num_kor{}'.format(i), 'dupl_list_kor{}'.format(i)]
#     tmp['num_kor{}'.format(i)] = a['stwd_kor']
    aa = a.merge(globals()['stwd_kor_uni{}'.format(i)], left_on = 'index', right_on = 'index', how = 'left')
    print('===========================================================================')
    print(a)
    df_index_tmp = df_index_tmp.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        left_on ='index',
        right_on = 'index',
        how = 'left'
    )
df_index_tmp

In [ ]:
df_index_tmp.columns

In [ ]:
tmp = pd.DataFrame()
for i in range(kor_firts_cnt):
#     a = pd.DataFrame()
    a = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     tmp['num_kor{}'.format(i)] = a['stwd_kor']
    tmp = df_index.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        left_on ='index',
        right_on = 'index',
        how = 'left'
    )
    print(i)
    print('===========================================================================')
    print(tmp)
tmp

In [ ]:
stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor0']

In [ ]:
stwd_total_kor_unique.loc[stwd_total_kor_unique['num_kor'] == 'num_kor0', 'num_kor']

In [ ]:
unique_stwd_kor['stwd_kor'].drop_duplicates()

In [ ]:
pd.DataFrame(unique_stwd_kor['stwd_kor'].drop_duplicates())

In [ ]:
# 특수기호 검사 : 한, 영
# 소문자 검사 : 한, 영
# 스페이스 제거 : 언제나
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")

# 참고

In [ ]:
############ num_kor(n) / num_eng(n) 기준으로 테이블 분리
############ index 기준으로 left join하여 원상복구

In [ ]:
############## 이거 참고
# db 조건 조회
print(len(stwd_eng7.loc[stwd_eng7['stwd_eng7'] != "", 'stwd_eng7']))
# 특수기호 제거
# for i in range(len(input_test['영문약어명'])):
#     tmp = input_test['영문약어명'][i].translate(str.maketrans('', '', string.punctuation))
#     print(tmp)
print(string.punctuation)

In [ ]:
# # 특수문자 제거
# for i in range(len(input_test['영문약어명'])):
#     tmp = input_test['영문약어명'][i].translate(str.maketrans('', '', string.punctuation))
#     i += i
#     print(tmp)
# i

In [ ]:
# # -*- coding: utf-8 -*-
# import sys
# import re

# def isHangul(text):
#     #Check the Python Version
#     pyVer3 =  sys.version_info >= (3, 0)

#     if pyVer3 : # for Ver 3 or later
#         encText = text
#     else: # for Ver 2.x
#         if type(text) is not unicode:
#             encText = text.decode('utf-8')
#         else:
#             encText = text

#     hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
#     return hanCount > 0

# 한글 영문 추리기1
# if __name__ == '__main__':
#     han = '한글'
#     han2 = u'한글'
#     input = 'English1234'  #utf-8
#     input2 = u'English1234' #unicode
#     #(major_ver, minor_ver) = (sys.version_info).split(',')
#     print(han + " is " + str(isHangul(han)))
#     print(han2 + " is " + str(isHangul(han2)))
#     print(input + " is " + str(isHangul(input)))
#     print(input2 + " is " + str(isHangul(input2)))

# 한글 영문 추리기2
# i = 23
# print(stwd_total['stwd_eng'][i])
# print(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', stwd_total['stwd_eng'][i]))
# print(re.findall('[A-Za-z]', stwd_total['stwd_eng'][i]))

In [ ]:
# # 영문명에 한글명 포함되었는가 탐색\
        
# # 한글 영어 최종 검색
# print('[자동 처리 시작합니다]')
# chk_list = []
# stwd_total['stwd_cor_kor'] = ''
# for i in range(len(stwd_total)):
#     cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total['stwd_kor'][i])
#     chk_kor = len(cor_kor) == len(list(stwd_total['stwd_kor'][i]))
   
#     if bool(chk_kor) == False:
#         print("-----------------------------------------------------------------|")
#         print('index: ' + str(i))
#         print(stwd_total['stwd_kor'][i] + " -> " + str(''.join(cor_kor)))
#         chk_list.append(i)
#     stwd_total['stwd_cor_kor'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total['stwd_kor'][i])

    
# #     stwd_total.loc[i,'stwd_kor_chk'] = cor_kor
# stwd_total_manu1 = stwd_total.iloc[chk_list]
# print('[자동 처리 완료하였습니다]')
# print("chk_list: "+ str(len(chk_list)))
# stwd_total_manu1

# # 특수기호 제거

# for i in range(stwd_total_cnt):
#     stwd_total.loc[i, 'stwd_kor'] =  stwd_total['stwd_kor'][i].translate(str.maketrans('', '', string.punctuation))
# #     stwd_total['stwd_kor'][i] = stwd_total['stwd_kor'][i].translate(str.maketrans('', '', string.punctuation)) # 안됨!!!
#     cnt += i
# print('[test] 적용 후: ' + str(stwd_total.loc[0, 'stwd_kor']))

In [ ]:
# #####################################################################
# # 한글 소문자 탐색 
# test = 'test'
# stwd_total.loc[2, 'stwd_kor'] = stwd_total.loc[2, 'stwd_kor'] + test
# print('[test] 적용 전(가짜): ' + stwd_total.loc[2, 'stwd_kor'])
# df_lower_kor = pd.DataFrame(columns = stwd_total.columns)
# for i in range(stwd_total_cnt):
#     if stwd_total['stwd_kor'][i].islower():
#         tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
#         df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
# stwd_total['stwd_kor_old'] = stwd_total['stwd_kor']
# stwd_total['stwd_kor'] = stwd_total['stwd_kor'].str.upper()

# # 적용 후 결과값
# print('[test] 적용 후(대문자): ' + stwd_total.loc[2, 'stwd_kor'])
# stwd_total.loc[2, 'stwd_kor'] = stwd_total.loc[2, 'stwd_kor'].strip(test.upper())
# print('[test] 복구: 개시')
# print('변경된 값: ' + str(len(df_lower_kor)) + '개')
# stwd_total.head()

# # 영문 소문자 탐색 
# print('[test] 적용 전(소문자): ' + stwd_total.loc[2, 'stwd_eng'])
# df_lower_eng = pd.DataFrame(columns = stwd_total.columns)
# for i in range(stwd_total_cnt):
#     if stwd_total['stwd_eng'][i].islower():
#         tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
#         df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
# stwd_total['stwd_eng_old'] = stwd_total['stwd_eng']
# stwd_total['stwd_eng'] = stwd_total['stwd_eng'].str.upper()

# # 적용 후 결과값
# print('[test] 적용 후(대문자): ' + stwd_total.loc[2, 'stwd_eng'])
# print('변경된 값: ' + str(len(df_lower_eng)) + '개')
# stwd_total.head()

In [ ]:
# left join
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )

# 카운트 일치 여부 점검
index_cnt = len(df_index['index'])
if index_cnt != len(stwd_total0):
    print("index 와 joined_date count불일치")
else:
    print("count 일치")
stwd_total5

In [ ]:
# merge
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.concat(
        [
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)]
        ],
        axis = 1        
    )
    print(i)
# print(name_cnt)
# print(name_cnt_diff_list)


stwd_total1

In [ ]:
for i in list(set(range(name_cnt)) - set(range(name_cnt_diff))):
    print(i)


In [ ]:
list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))

In [ ]:
input_split_eng

In [ ]:
input_split_eng['stwd_eng6'] = None
input_split_eng['stwd_eng6']

In [ ]:
input_split_eng['stwd_eng6']

In [ ]:
# 한글/영문 컬럼 수 비교하기
name_cnt_kor = len(name_list_kor)
name_cnt_eng = len(name_list_eng)

if name_cnt_kor == name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = None
elif name_cnt_kor > name_cnt_eng:
    name_cnt = name_cnt_eng
    name_cnt_diff_list = list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))
    for i in name_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)]),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)])
            ],
            axis = 1
        )
        
        
        
elif name_cnt_kor < name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = list(set(range(name_cnt_eng)) - set(range(name_cnt_kor)))
    for i in name_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)]),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)])
            ],
            axis = 1
        )
        

stwd_eng7


In [ ]:
stwd_eng6

In [ ]:
# 한글/영문 컬럼 수 비교하기
name_cnt_kor = len(name_list)
name_cnt_eng = len(name_list_eng)

if name_cnt_kor == name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = None
elif name_cnt_kor > name_cnt_eng:
    name_cnt = name_cnt_eng
    name_cnt_diff_list = list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))
    for i in name_cnt_diff_list:
        globals()['stwd_eng{}'.format(i)] = None
elif name_cnt_kor < name_cnt_eng:
    name_cnt = name_cnt
    name_cnt_diff_list = list(set(range(name_cnt_eng)) - set(range(name_cnt_kor)))
    
    

    
    
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.concat(
        [
        globals()['stwd{}'.format(i)],
        globals()['stwd_eng{}'.format(i)]
        ],
        axis = 1        
    )
    print(i)
# print(name_cnt)
# print(name_cnt_diff_list)

In [ ]:
# 리스트 - 리스트
list1 = [1,2]
list2 = [1,2,3]
list(set(list2) - set(list1))
list(set(list1) - set(list2))
list(set(list1).difference(set(list2)))
list(set(range(10)) - set(range(2)))

In [ ]:
list(set(range(10)) - set(range(2)))

In [ ]:
list(set(range(2)) - set(range(10)))

In [ ]:
# 용어명 분리
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
name_list = []
for i in range(len(input_split.columns)):
    name_list.append("stwd{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = name_list)
# 인덱스 컬럼 추가
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']
input_split= pd.concat([input_split, df_index], axis = 1)
print(input_split.shape)
input_split.head(5)

In [ ]:
# 영문명 분리
input_test['영문약어명'] = input_test['영문약어명'].str.replace(" ", "")
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist())
name_list_eng = []
for i in range(len(input_split_eng.columns)):
    name_list_eng.append("stwd_eng{}".format(i))
print(name_list_eng)
# 컬럼명 만들기
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist(), columns = name_list_eng)
# 인덱스 컬럼 추가
# df_index = pd.DataFrame(input_test['No'])
# df_index.columns = ['index']
# input_split_eng= pd.concat([input_split_eng, df_index], axis = 1)
print(input_split_eng.shape)
input_split_eng.head(5)

In [ ]:
name_list = []
for i in range(len(input_split_eng.columns)):
    name_list_eng.append("stwd_eng{}".format(i))

In [ ]:
if len(name_list) == len(name_list_eng):
    name_list_cnt = len(name_list)
elif len(name_list) > len(name_list_eng):
    name_list_cnt = len(name_list)
elif len(name_list) < len(name_list_eng):
    name_list_cnt = len(name_list_eng)
    
for i in range(name_list_cnt):
    print(i)

In [ ]:
if len(name_list) == len(name_list_eng):
    print(1)
elif len(name_list) > len(name_list_eng):
    print(2)    
elif len(name_list) < len(name_list_eng):
    print(3)

In [ ]:
len(name_list_eng)

In [ ]:
len(name_list)

In [ ]:
# 용어명 분리
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
name_list = []
for i in range(len(input_split.columns)):
    name_list.append("stwd{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = name_list)
print(input_split.shape)
input_split.head(5)

In [ ]:
# 단어별 개수 세기(네이밍 컬럼)
dupl_list = []
dupl_cnt = []
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []
    for j in range(len(input_split["stwd{}".format(i)])):
        dupl_cnt = input_split["stwd{}".format(i)].to_list().count(input_split["stwd{}".format(i)][j])
        globals()["dupl_list{}".format(i)].append(dupl_cnt)
# df_dupl 힌즐 세우기
df_dupl = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)]
    df_dupl = pd.concat([df_dupl, pd.DataFrame(globals()["dupl_list{}".format(i)])], axis = 0)
df_dupl.columns = ['dupl']
df_dupl

In [ ]:
# df_index 한줄 세우기
# df_index = pd.DataFrame(input_test['No'])
df_index = pd.DataFrame()
for i in range(len(input_split.columns)):
#     globals()["df_index{}".format(i)] = pd.DataFrame(input_test['No'])
    df_index = pd.concat([df_index, pd.DataFrame(input_test['No'])])
df_index.columns = ['index']
df_index

In [ ]:
# df_stwd 힌즐 세우기
df_stwd = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["df_stwd{}".format(i)] = input_split["stwd{}".format(i)]
    df_stwd = pd.concat([df_stwd, globals()["df_stwd{}".format(i)]])
df_stwd.columns = ['stwd']
df_stwd

In [ ]:
# df_chk
df_chk = pd.concat([df_stwd, df_index], axis=1).reset_index() # 인덱스 초기화
df_chk = df_chk[['stwd', 'index']]
print(df_chk.shape)
df_chk.tail(5)

In [ ]:
### 한글 단어 분리

In [ ]:
# 영문명 분리
input_test['영문약어명'] = input_test['영문약어명'].str.replace(" ", "")
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist())
name_list = []
for i in range(len(input_split_eng.columns)):
    name_list.append("stwd_eng{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist(), columns = name_list)
print(input_split_eng.shape)
input_split_eng.head(5)

In [ ]:
# df_index 한줄 세우기
df_index_eng = pd.DataFrame()
for i in range(len(input_split_eng.columns)):
    df_index_eng = pd.concat([df_index_eng, pd.DataFrame(input_test['No'])])
df_index_eng.columns = ['index']
df_index_eng

In [ ]:
# df_stwd_eng 힌즐 세우기
df_stwd_eng = pd.DataFrame()
for i in range(len(input_split_eng.columns)):
    globals()["df_stwd_eng{}".format(i)] = input_split_eng["stwd_eng{}".format(i)]
    df_stwd_eng = pd.concat([df_stwd_eng, globals()["df_stwd_eng{}".format(i)]])
df_stwd_eng.columns = ['stwd_eng']
df_stwd_eng

In [ ]:
# df_chk_eng
df_chk_eng = pd.concat([df_stwd_eng, df_index_eng], axis=1).reset_index() # 인덱스 초기화
df_chk_eng = df_chk_eng[['stwd_eng', 'index']]
print(df_chk_eng.shape)
df_chk_eng.tail(5)

In [ ]:
### 영문 단어 분리

In [ ]:
df_chk.merge(df_chk_eng, on='index' 

In [ ]:
# df_chk_uni
df_chk_uni = df_chk[df_chk['stwd'].duplicated()==False].reset_index()
df_chk_uni = df_chk_uni[['stwd', 'index']]
df_chk_uni.tail(5)

In [ ]:
# unique 단어 개수 세기
dupl_list = []
dupl_cnt = []
for i in range(len(df_chk_uni["stwd"])):
    dupl_cnt = df_chk_uni["stwd"].to_list().count(df_chk_uni["stwd"][i])
    dupl_list.append(dupl_cnt)

len(dupl_list)

In [ ]:
# 횟수 병합
df_chk_uni = pd.concat([df_chk_uni, pd.DataFrame(dupl_list)], axis = 1)
df_chk_uni.columns = ['stwd', 'index', 'dupl_cnt']
print(df_chk_uni[df_chk_uni['dupl_cnt'] != 1])
print(df_chk_uni[df_chk_uni['dupl_cnt'] == None])
df_chk_uni.tail(5)

In [ ]:
# 점검 , 변환
# ab['dupl_cnt'].info()
# ab['dupl_cnt'].astype(int)

In [ ]:
# df_chk_uni
df_chk_dupl = df_chk[df_chk['stwd'].duplicated()].reset_index()
df_chk_dupl = df_chk_dupl[['stwd', 'index']]
df_chk_dupl.tail(5)

In [ ]:
# dupl 단어 개수 세기
dupl_list = []
dupl_cnt = []
for i in range(len(df_chk_dupl["stwd"])):
    dupl_cnt = df_chk_dupl["stwd"].to_list().count(df_chk_dupl["stwd"][i])
    dupl_list.append(dupl_cnt)

len(dupl_list)

In [ ]:
# 횟수 병합
df_chk_dupl = pd.concat([df_chk_dupl, pd.DataFrame(dupl_list)], axis = 1)
df_chk_dupl.columns = ['stwd', 'index', 'dupl_cnt']
print(df_chk_dupl[df_chk_dupl['dupl_cnt'] == 1])
print(df_chk_dupl[df_chk_dupl['dupl_cnt'] == None])
df_chk_dupl.tail(5)

In [ ]:
df_chk_dupl[df_chk_dupl['stwd'] == '매수']

In [ ]:
cc = df_chk_dupl.merge(input_test, left_on = 'index', right_on = 'No', how='left')
cc = cc[cc['stwd'].duplicated()==False]
print(cc.shape)
cc.head(3)

In [ ]:
df_chk_dupl.to_excel(input_path+"dsfadas.xlsx")

In [ ]:
cc = df_chk_dupl['stwd'].drop_duplicates()
(cc.shape)
cc

In [ ]:
cc.to_excel(input_path+"cccccccccccc.xlsx")

In [ ]:
df_chk_dupl['stwd'].drop_duplicates()

In [ ]:
ccc = df_chk_dupl.merge(input_test, left_on = 'index', right_on = 'No', how='left')
ccc = ccc[ccc['stwd'].duplicated()]
print(ccc.shape)
ccc.head(3)

In [ ]:
ccc.to_excel(input_path+"cccccccccc3cc.xlsx")

In [ ]:
a[a['stwd'].duplicated()]

In [ ]:
a[pd.DataFrame(a['stwd']).duplicated(keep='first')]

In [ ]:
a['stwd'].drop_duplicates()

In [ ]:
a = pd.concat([df_stwd, df_dupl, df_index], axis=1)
a

In [ ]:
tOK = a.loc[a['dupl'] == 1]
# tOK['stwd'].dropna()
tOK[tOK['stwd'] == '검토']
tOK
tNO = a.loc[a['dupl'] > 1]
# tNO['stwd'].dropna()
tNO[tNO['stwd'] == '검토']
tNO

In [ ]:
tOK['stwd'].drop_duplicates()

In [ ]:
print(368 + 412)
tNO['stwd'].drop_duplicates()

In [ ]:
a['stwd'].drop_duplicates()

In [ ]:
t2['stwd'].drop_duplicates()

In [ ]:
a.to_excel(input_path+"\\dfdf.xlsx")

In [ ]:
tOK.to_excel(input_path+"\\dfdfOK.xlsx")

In [ ]:
tNO.to_excel(input_path+"\\dfdfNO.xlsx")

In [ ]:
tNO['stwd'].drop_duplicates().to_excel(input_path+"\\dfdfNO_DUP.xlsx")

In [ ]:
t2['stwd'].drop_duplicates().tail(30)

In [ ]:
# t2 = a[a['stwd'].duplicated(keep='first')]
t2 = a[a['stwd'].duplicated(False)]
t2[t2['stwd'] == '검토']
# t2['stwd'].dropna()
t2

In [ ]:
t2.drop_

In [ ]:
a[a['stwd'] == '검토']

In [ ]:
a.head(20)

In [ ]:
a['stwd'].duplicated(keep='first').head(20)
# t#['stwd'].dropna()
# t3[t3['stwd'] == '검토']

In [ ]:
# # concat 을 위한 DataFame 묶기 (o 시리즈여야 한곳에 정렬됨! 데이터 프레임 안됨)
# for i in range(len(input_split.columns)):
#     globals()["df_stwd{}".format(i)] = ["duple_list{}".format(i)]
# df_stwd0

In [ ]:
# concat 을 위한 DataFame 묶기 (o 시리즈여야 한곳에 정렬됨! 데이터 프레임 안됨)
for i in range(len(input_split.columns)):
    globals()["df_stwd{}".format(i)] = input_split["stwd{}".format(i)]
df_stwd0

In [ ]:
# df_index 묶기
for i in range(len(input_split.columns)):
    globals()["df_index{}".format(i)] = input_split["stwd{}".format(i)]
df_stwd0

In [ ]:
df_stwd = pd.concat([df_stwd, df_index], axis=1)
df_stwd

In [ ]:
pd.concat([input_split, pd.DataFrame(input_test['No'])], axis=1)


In [ ]:
# df_stwd.tail(500)

In [ ]:
list = []
for i in range(len(input_split.columns)):
    df = pd.concat([globals()["df_stwd{}".format(i)]])
df

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    df = pd.concat([df, globals()["df_stwd_df{}".format(i)]])
df 

In [ ]:
df

In [ ]:
df_dupl_cnt = pd.DataFrame()
for in in range(len(input_split.coulmns):
    df_dupl_cnt = pd.concat([df, globals()["df{}".format(i)]])

In [ ]:
dupl_list0

In [ ]:
# 컬럼 재정렬
list = []
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]
    list.append("stwd{}".format(i))
    list.append("dupl_cnt{}".format(i))
input_split = input_split[list]
input_split

In [ ]:
df_stwd = pd.DataFrame()
for i in range(len(input_split.columns)):
    df_stwd = pd.concat([df, globals()["df{}".format(i)]])
df_stwd 

In [ ]:
pd.DataFrame

In [ ]:
type(input_split["stwd{}".format(0)])

In [ ]:
a = pd.Series
a

In [ ]:
a

In [ ]:
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = input_split["stwd{}".format(i)]
    list.append('df{}'.format(i))
list

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = pd.DataFrame(input_split["stwd{}".format(i)])
#     list.append('df{}'.format(i))
# list

In [ ]:
list = []
for i in range(len(input_split.columns)):
    list.append("df{}".format(i))
list

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    df = pd.concat([df, globals()["df{}".format(i)]])
df

In [ ]:
concat([df0, df0,df0,df0,df0,df0,df0,df0,])

In [ ]:
list = ",".join(list)
list
list[5]

In [ ]:
",".join(list[0])

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    ccc = pd.concat(df, ",".join(list[i]))
df = pd.DataFrame()
df = pd.concat([df, df0])

In [ ]:
df = pd.DataFrame()
df = pd.concat([df, df0])

In [ ]:
",".join(list)

In [ ]:
list = ",".join(list)
list

In [ ]:
list

In [ ]:
df0

In [ ]:
list = [df0, df1, df2, df3, df4, df5]
pd.concat(list)

In [ ]:
pd.concat([df0, df1, df2, df3, df4, df5])


In [ ]:
input_split

In [ ]:
stwd_list

In [ ]:
stwd_list = input_split.columns.to_list()
for i in range(len(input_split.columns)):
    duplicated = pd.concat(stwd_list, ignore_index=True)

In [ ]:
stwd_list = input_split.columns.to_list()
pd.concat(pd.DataFrame(input_split[stwd_list]), ignore_index = True)

In [ ]:
list = []
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = input_split["stwd{}".format(i)]
    list.append(globals()["df{}".format(i)])

In [ ]:
pd.DataFrame(list)

In [ ]:
pd.DataFrame(input_split["stwd1"])

In [ ]:
input_split["stwd0"]

In [ ]:
pd.concat(pd.DataFrame(input_split["stwd0"]), pd.DataFrame(input_split["stwd1"]))

In [ ]:
df0
list

In [ ]:
globals("df0")

In [ ]:
pd.concat([df0, df1])

In [ ]:
pd.concat([df0, df1])

In [ ]:
for i in range(len(input_split.columns)):
#     pd.concat(globllist[i])
    print(list[i])

In [ ]:
for i in range(len(input_split.columns)):
#     pd.concat(["df{}".format(i)])
#     pd.concat("df{}".format(i))
    pd.concat(list)  
#     print("df{}".format(i))

In [ ]:
for i in range(len(input_split.columns)):
    print("df{}".format(i))

In [ ]:
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = pd.DataFrame(globals()["df{}".format(i)])

In [ ]:
list = []
for i in range(len(input_split.columns)):
    list.append("df{}".format(i))

In [ ]:
df1

In [ ]:
pd.concat(list, axis=1 )
#           ignore_index=True)

In [ ]:
for i in range(len(input_split.columns)):
    pd.concat(globals()["df{}".format(i)])

In [ ]:
pd.concat([df0, df1])

In [ ]:
stwd_list = input_split.columns.to_list()
for i in range(len(input_split.columns)):
    pd.concat(pd.DataFrame(input_split["stwd{}".format(i)]))
list

In [ ]:
input_split["stwd0"]

In [ ]:
pd.DataFrame(input_split["stwd{}".format(0)])

In [ ]:
cc = pd.DataFrame(input_split["stwd{}".format(i)])

In [ ]:
aa = pd.DataFrame
for i in range(len(input_split.columns)):
    aa = aa.append(cc)
aa

In [ ]:
for i in range(len(input_split.columns)):
    a = pd.concat([input_split[list[i]]])
a

In [ ]:
a

In [ ]:
pd.concat(list)

In [ ]:
for i in range(len(input_split.columns)):
    pd.concat([globals["input_split{}".format(i)]])

In [ ]:
# 컬럼 재정렬
list = []
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]
    list.append("stwd{}".format(i))
    list.append("dupl_cnt{}".format(i))
input_split = input_split[list]
input_split

In [ ]:
input_split.columns

In [ ]:
for i in range(int(len(input_split.columns)/2)):
    aaa= pd.concat(["stwd{}".format(i)])
aaa

In [ ]:
input_split[0]

In [ ]:
input_split[0]

In [ ]:
input_split[0].count("가입")

In [ ]:
dupl_list =[]
dupl_list

In [ ]:
len(input_split.columns)

In [ ]:
input_split[0].to_list().count(input_split[0][11])

In [ ]:
globals()["stwd{}".format(i)]

In [ ]:
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []

In [ ]:
input_split[0].to_list().count(input_split[0][3])

In [ ]:
# 성공
dupl_list = []
dupl_cnt = []


for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []
    for j in range(len(input_split[i])):
        dupl_cnt = input_split[i].to_list().count(input_split[i][j])
        globals()["dupl_list{}".format(i)].append(dupl_cnt)
#         print(dupl_cnt)
#         print("dupl_list{}".format(i))




In [ ]:
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]

In [ ]:
input_split["dupl_cnt0"]

In [ ]:
# input_split
input_split[input_split["dupl_cnt0"] >= 2][["dupl_cnt0","dupl_cnt1",0]]

In [ ]:
input_split.loc[:,"dupl_cnt1"] = dupl_list1

In [ ]:
input_split

In [ ]:
for i in range(len(input_split.columns)):
    print(globals()["dupl_list{}".format(i)])

In [ ]:
a= input_split.assign(dd =  dupl_list3)
# df.insert(3,"city",['Lahore','Dehli','New York'],True)
a

In [ ]:
input_split.insert(8,"dd",dupl_list3,True)
input_split

In [ ]:
input_split[2].to_list().count('역')

In [ ]:
input_split[0].to_list()

In [ ]:
dupl_list3

In [ ]:
input_split.insert(8,"dd",dupl_list3,True)

In [ ]:
input_split

In [ ]:
pd.DataFrame(input_split[3].to_list(), dupl_list3)

In [ ]:
print(len(dupl_list))
dupl_list

In [ ]:
# type(input_split[0])
input_split[0].to_list().count('가입')

In [ ]:
# 성공
b = []
for i in range(len(input_split[0])):
    a = 0
    for j in input_split[0]:
        b.append(a)
        if input_split[0][i] == j:
            a += 1
            print(input_split[0][i] +"&&"+j+"_중복")
           
            print(a)


In [ ]:
len(b)

In [ ]:
# 성공
a = 0
for i in range(len(input_split[0])):
    for j in input_split[0]:
        if input_split[0][i] == j:
            print(input_split[0][i] +"&&"+j+"_중복")
            a += 1
            print(a)


In [ ]:
# 성공
a =0
for i in input_split[0]:
    for j in input_split[0].unique():#중복 
        if i == j:
            print(i +"&&"+j+"_중복")
            a += 1
            print(a)
#         else:
#             print(i+j)
    
    
    
#     if i == '가입':
#         print(i+"_성공")
#     else:
#         print(i)

In [ ]:
for i in input_split[0]:
    if i == '가입':
        print(i+"_성공")
    else:
        print(i)

In [ ]:
input_split[0].str == "가입"

In [ ]:
for i in a
    input_split[0] == i
    

In [ ]:
input_split[0]

In [ ]:
a = input_split[0].unique()
a

In [ ]:
len(input_split.columns)

In [ ]:
input_processing = pd.concat([input_split, input_test])
print(input_processing.columns)
input_processing.head(5)

In [ ]:
input_process["No"]

In [ ]:
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : row[10])
# # stwd1

In [ ]:
aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = ['a' ,'b', 'c', 'd','a' ,'b', 'c', 'd'])
aa

In [ ]:
aaa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
aaa

In [ ]:
aaa[1]

In [ ]:
aaa.columns

In [ ]:
aa['a']

In [ ]:
aa.columns

In [ ]:
for i in range(len()):
    aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = globals()["stwd{}".format(i)])
    aa

In [ ]:

try:    
    for i in range(10):
         globals()["stwd{}".format(i)] = pd.DataFrame({"stwd{}".format(i) : []})
except:
    pass
# print(stwd0)

    
        
    
# pd.DataFrame({"stwd0" : []})
        

In [ ]:
print(input_test['용어명 분리'].str.split('_', maxsplit=1))

In [ ]:
# try:
for i in range(10):
    globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_').map(lambda row : row[i])

input_test['용어명 분리'].str.split('_').map(lambda row : row[0])

In [ ]:
pd.DataFrame(input_test['용어명 분리'].str.split('_'))


In [ ]:
input_test['용어명 분리'].str.split('_').tolist() 

In [ ]:
aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = ['a' ,'b', 'c', 'd','a' ,'b', 'c', 'd'])
aa

In [ ]:
input_test['용어명 분리'].str.split('_').map(lambda x:x[-1])


In [ ]:
a= pd.DataFrame(input_test['용어명 분리'].str.split('_'))
a

In [ ]:
a.str.get(0)

In [ ]:
input_test['용어명 분리'].str.split('_')[0][0]


In [ ]:
input_test['용어명 분리'].str.split('_').map(lambda row : row[-1])

In [ ]:
stwd

In [ ]:
stwd2

In [ ]:
# i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
#     globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_', maxsplit = -1).map(lambda row : row[i])

# stwd0
# stwd1

# stwd0 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[0]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd2 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd3 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd4 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd5 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd6 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd7 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd8 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd9 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
try:
    stwd10 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[10]))
except:
    pass


In [ ]:
# i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
num = 10
for i in range(10):
#     if input_test['용어명 분리'].str.split("_").map(lambda row : row[i]) 
    
    globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_', maxsplit = -1).map(lambda row : row[i])

stwd0
# stwd1


# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))


In [ ]:
stwd1

In [ ]:
input_f = os.path.join(os.getcwd(), "input")
print(input_f)
# pd.read_excel(input_f+"test.xlsx", sheet_name="input_test", header=0)
sheet_name = "input_test"
pd.read_excel("C:/test.xlsx", sheet_name=sheet_name, header=0)

# current_path = os.path.dirname(input_f)
# input_f = os.path.join(current_path, "input")
print()
# current_path = os.path.dirname()
# print(current_path)
print(os.getcwd())

In [ ]:
print(os.getcwd())
# print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))

In [ ]:
def a(b, *values):
    a = [*values]
    print(b)
    print(a)
a("dfd",1,2,3,5)

[*range(len(input_meta_col))]

In [ ]:
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
        print(type(col))

In [ ]:
# null/입력 값 검색 로직

def nll_chk(data):
    cnt_chk = len(data)

    chk_nll_list = []
    chk_nll_cnt_list = []
    # null값 체크(개수 파악)
    for i,j in zip(range(len(data.columns)), data.columns):
        indv_cnt = data.loc[data[j] != '', j]
        if len(data[j]) != len(indv_cnt):
            print(j + ": " + str(len(indv_cnt)))
            chk_nll_list.append(j)
            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)
    print("================================")
    if len(chk_nll_list) == 0:
        print("모든 값이 입력되었습니다.")
    elif len(chk_nll_list) > 1 :
        print("null이 포함된 컬럼이 탐지되었습니다.")
        print(chk_nll_list)
        print("================================")

    globals()["cnt_chk_out"] = pd.DataFrame(zip(chk_nll_list, chk_nll_cnt_list), columns = ["NULL포함컬럼명", "NULL개수"])
    print(cnt_chk_out)
    
    print("================================")
    print("null 인덱스를 임시 보관합니다.")
    print("================================")
    nll_chk_row = cnt_chk_out.iloc[:,0].to_list()
    for i, j in zip(range(len(nll_chk_row)), nll_chk_row):
        globals()["null_{}".format(i)] = data.loc[data[j] == '', j].index
        print("널인덱스보관_" + str(j) + "(변수명) -> null_{}".format(i))
        
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
#         print(type(col))

# 불필요 컬럼 빼기 - 이름으로 빼기
def brief_col(data, excpt_col_list, name):
    col_list = data.columns
    selected_col_list = [x for x in col_list if x not in excpt_col_list]
    print(selected_col_list)
    globals()["{}".format(name)] = data[selected_col_list]
    
# 불필요 컬럼 빼기 - 순서로 빼기 
# tmp_col_list = [5,6,7,8]
# tmp_tbl_list = [2,3,4,5]
def brief_col_num(data, excpt_col_list, name):
    col_list = data.columns
    col_list_num = [*range(len(col_list))]
    selected_col_list_num = [x for x in col_list_num if x not in excpt_col_list]
    selected_col = col_list[selected_col_list_num]
    print(selected_col)
    globals()["{}".format(name)] = data[selected_col]

# 컬럼명(순서) 선택 및 변경
def cor_col(data, select_col, modify_col, name):
    # 컬럼명(순서) 선택
    globals()["{}".format(name)] = data[select_col]
    # 컬렴명 변경
    globals()["{}".format(name)].columns = modify_col
    
# null 공백처리 함수
def nll_to_blnk(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 Null 공백 처리]")
        for i in data.columns:
            data[i] = data[i].fillna("")
            print(str(i) + ":\t Null 공백 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].fillna("")
        print(col + ":\t  공백 처리 성공")

# 컬럼 개수 비교
def cmpar(data, col1, col2):
    crtr_1 = len(data[col1].unique()) - 1
    crtr_2 = len(data[col2].unique()) - 1
    if crtr_1 == crtr_2:
        print('[정상] 용어명과 분리된용어명의 수가 같습니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
    else:
        print('[오류] 용어명과 분리된용어명의 수가 다릅니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
        
# 공백 null처리 함수
def blank_to_nll(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 공백 Null 처리]")
        for i in data.columns:
            data.loc[data[i] == '', i] = None
            print(str(i) + ":\t Null 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data.loc[data[col] == '', col] = None
        print(col + ":\t  Null 처리 성공")
        
def row_nll_rmv(data, name):
    ### 값이 한개라도 있을 경우만 카운트 ###

    data_cnt = len(data)
    notna_list = []
    for i in range(data_cnt):    
        # row 기준 널 체크(값이 한개라도 있을 경우 포함)
        if data.iloc[i].notna().sum() > 1:
            notna_list.append(i)
    globals()["{}".format(name)] = globals()["{}".format(data)].loc[notna_list]
    print(notna_list)
    print(data.shape)